In [1]:
#@title Installations préalables

!pip install -U -q "google"
!pip install -U -q "google.genai"

import os
from google.colab import userdata
from google.colab import drive
os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import shutil
import os

src = '/content/drive/MyDrive/folder_partitioned_merged/Le bonheur est-il affaire de hasard et de nécessité ? (AGREG interne -  note : 15,5)'
dst = '/content/Le bonheur est-il affaire de hasard et de nécessité 15.5 ? (AGREG interne -  note : 15,5)'

if os.path.exists(dst):
    shutil.rmtree(dst)

shutil.copytree(src, dst)

'/content/Le bonheur est-il affaire de hasard et de nécessité 15.5 ? (AGREG interne -  note : 15,5)'

In [2]:
import shutil
import os

src = '/content/drive/MyDrive/folder_partitioned_merged_clean'
dst = '/content/main_folder'

if os.path.exists(dst):
    shutil.rmtree(dst)

shutil.copytree(src, dst)

'/content/main_folder'

In [ ]:
#@title OCR SERIAL LOOP (26/04/25)

import os
import google.generativeai as genai
import csv
import time
import re
from tqdm.notebook import tqdm
import traceback

# --- Configuration ---
MAIN_FOLDER_PATH = "/content/main_folder"  # Main folder containing subfolders
os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")


try:
    genai.configure(api_key=os.environ["GEMINI_API_KEY_3"])
    print("Gemini API Key configured.")
except KeyError:
    print("🛑 Error: GEMINI_API_KEY environment variable not set.")
except Exception as e:
    print(f"🛑 Error configuring Gemini API: {e}")

# Gemini Model Configuration
GENERATION_CONFIG = {
  "temperature": 0.2,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}
MODEL_NAME = "gemini-2.5-pro-preview-03-25"

# System Prompts (remain the same)
SYSTEM_INSTRUCTION_WITH_CONTEXT = """Tâche Principale : Recopie aussi fidèlement que possible le texte manuscrit de l'image fournie. Répond **exclusivement** avec le texte transcrit, sans aucune introduction, commentaire, ou note.

Contexte de la Dissertation : Ce texte est un extrait d'une dissertation de philosophie portant sur le sujet "{sujet_copie}".

Mise en forme :
- Si des mots ou groupes de mots sont soulignés dans l'image, mets-les en italique en markdown (*texte souligné*).
- Si un mot est barré, encadre-le de tildes en markdown (~mot barré~).

Aide Contextuelle (Texte Précédent) : Pour t'aider à déchiffrer l'écriture, le texte qui précède *immédiatement* celui de l'image actuelle est fourni ci-dessous (séparé par '--- Texte Précédent ---'). Ce texte a été obtenu par OCR des sections précédentes de la dissertation. Le texte de l'image actuelle commence là où le texte précédent s'arrête (il peut y avoir un léger chevauchement de lignes dû au découpage des images).

**Priorité :** Ta tâche principale reste de transcrire *exactement* ce que tu vois sur l'image actuelle. Utilise le texte précédent **uniquement** comme une aide *secondaire* pour résoudre des ambiguïtés ou déchiffrer des mots très difficiles. **Ne laisse PAS le texte précédent remplacer ou modifier ce qui est clairement visible sur l'image actuelle.** En cas de conflit entre le texte précédent et l'image, **la transcription fidèle de l'image PRÉVAUT.**
"""

SYSTEM_INSTRUCTION_FIRST_CHUNK = """Tâche Principale : Recopie aussi fidèlement que possible le texte manuscrit de l'image fournie. Répond **exclusivement** avec le texte transcrit, sans aucune introduction, commentaire, ou note.

Contexte de la Dissertation : Ce texte est un extrait d'une dissertation de philosophie portant sur le sujet "{sujet_copie}". Ceci est le tout début de la dissertation.

Mise en forme :
- Si des mots ou groupes de mots sont soulignés dans l'image, mets-les en italique en markdown (*texte souligné*).
- Si un mot est barré, encadre-le de tildes en markdown (~mot barré~).
"""

# --- Functions ---

def extract_folder_info(folder_path):
    """Extracts the full folder name while preserving all characters."""
    if not os.path.isdir(folder_path):
        return None, None
    full_name = os.path.basename(folder_path)
    return full_name, full_name  # Return full name for both values

# --- Function to sanitize filename ---
def sanitize_filename(filename):
    """Removes or replaces characters invalid for filenames."""
    # Remove leading/trailing whitespace
    sanitized = filename.strip()
    # Replace common invalid characters with underscore (adjust as needed)
    sanitized = re.sub(r'[\\/*?:"<>|]', '_', sanitized)
    # Replace forward slash specifically (common in paths)
    sanitized = sanitized.replace('/', '_')
    # Optional: Limit length if necessary (e.g., 200 chars)
    # max_len = 200
    # if len(sanitized) > max_len:
    #     name, ext = os.path.splitext(sanitized)
    #     sanitized = name[:max_len - len(ext) - 1] + '~' + ext
    return sanitized


# ... (upload_to_gemini, delete_gemini_file, natural_sort_key remain the same) ...
def upload_to_gemini(path, mime_type="image/png"):
    """Uploads a file to Gemini and returns the File object."""
    try:
        file = genai.upload_file(path, mime_type=mime_type)
        return file
    except Exception as e:
        print(f"  🛑 Error uploading file {path}: {e}")
        return None

def delete_gemini_file(file_object):
    """Deletes a file from Gemini service."""
    if file_object:
        try: genai.delete_file(file_object.name)
        except Exception as e: print(f"  ⚠️ Warning: Failed to delete file {file_object.name}: {e}")

def natural_sort_key(s):
    """Key for sorting strings containing numbers naturally."""
    return [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', s)]

# --- Main Processing Function ---
def process_folder(folder_path):
    if not os.path.isdir(folder_path):
        print(f"🛑 Error: Target folder not found at '{folder_path}'")
        return

    print(f"\nStarting OCR process for folder: '{os.path.basename(folder_path)}'")

    # 1. Extract folder info
    sujet_copie, nom_complet_sous_dossier = extract_folder_info(folder_path)
    if not sujet_copie:
        print(f"🛑 Error: Couldn't extract info from folder path '{folder_path}'")
        return

    print(f"  Subject detected: '{sujet_copie}'")

    # Generate output path
    parent_dir = os.path.dirname(folder_path)
    sanitized_full_name = sanitize_filename(nom_complet_sous_dossier)
    OUTPUT_CSV_FILE = os.path.join(parent_dir, f"{sanitized_full_name}.csv")
    print(f"  Output CSV will be: '{OUTPUT_CSV_FILE}'")

    try:
        with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([nom_complet_sous_dossier])

            # Find and sort images
            try:
                all_files = [f for f in os.listdir(folder_path) if f.lower().endswith(".png")]
                sorted_chunks = sorted(all_files, key=natural_sort_key)
            except OSError as e:
                print(f"  🛑 Error listing files: {e}")
                sorted_chunks = []

            if not sorted_chunks:
                print("  No PNG files found.")
                return

            print(f"  Found {len(sorted_chunks)} chunks to process.")
            ocr_result_n_minus_1 = ""
            ocr_result_n_minus_2 = ""

            for i, chunk_filename in enumerate(tqdm(sorted_chunks, desc="Processing Chunks")):
                chunk_path = os.path.join(folder_path, chunk_filename)
                gemini_file = None
                current_ocr_result = ""

                try:
                    # System prompt selection
                    if i == 0:
                        system_instruction = SYSTEM_INSTRUCTION_FIRST_CHUNK.format(sujet_copie=sujet_copie)
                        prompt_parts = []
                    else:
                        system_instruction = SYSTEM_INSTRUCTION_WITH_CONTEXT.format(sujet_copie=sujet_copie)
                        context_text = "\n--- Texte Précédent ---\n"
                        if ocr_result_n_minus_2:
                            context_text += ocr_result_n_minus_2 + "\n"
                        if ocr_result_n_minus_1:
                            context_text += ocr_result_n_minus_1
                        context_text += "\n--- Fin du Texte Précédent ---\n\n--- Image Actuelle à Transcrire ---"
                        prompt_parts = [context_text]

                    # Upload and process image
                    gemini_file = upload_to_gemini(chunk_path)
                    if not gemini_file:
                        current_ocr_result = f"ERROR: Upload failed {chunk_filename}"
                        writer.writerow([current_ocr_result])
                        continue

                    prompt_parts.append(gemini_file)
                    model = genai.GenerativeModel(
                        model_name=MODEL_NAME,
                        generation_config=GENERATION_CONFIG,
                        system_instruction=system_instruction,
                    )

                    # API call
                    response = model.generate_content(prompt_parts, request_options={'timeout': 600})

                    if response and response.parts:
                        current_ocr_result = response.text.strip()
                    else:
                        current_ocr_result = "ERROR: Empty response"

                    writer.writerow([current_ocr_result])

                    # Update history
                    ocr_result_n_minus_2 = ocr_result_n_minus_1
                    ocr_result_n_minus_1 = current_ocr_result

                except Exception as e_api:
                    print(f"🛑 Error processing {chunk_filename}:")
                    traceback.print_exc()
                    writer.writerow([f"ERROR: {type(e_api).__name__}"])

                finally:
                    if gemini_file:
                        delete_gemini_file(gemini_file)
                    time.sleep(1.5)

    except IOError as e:
        print(f"🛑 CSV Error: {e}")
    except Exception as e_main:
        print(f"🛑 Unexpected error:")
        traceback.print_exc()

    print(f"\nCompleted processing for: {os.path.basename(folder_path)}")
    print(f"CSV saved at: {OUTPUT_CSV_FILE}")

# --- Main Execution ---
if __name__ == "__main__":
    if not os.path.isdir(MAIN_FOLDER_PATH):
        print(f"🛑 Main folder not found: {MAIN_FOLDER_PATH}")
    else:
        print(f"\nStarting batch processing for: {MAIN_FOLDER_PATH}")
        subfolders = [os.path.join(MAIN_FOLDER_PATH, d)
                     for d in os.listdir(MAIN_FOLDER_PATH)
                     if os.path.isdir(os.path.join(MAIN_FOLDER_PATH, d))]

        subfolders.sort(key=natural_sort_key)  # Natural sort subfolders

        print(f"Found {len(subfolders)} subfolder(s) to process")
        for idx, subfolder in enumerate(subfolders, 1):
            print(f"\n=== Processing subfolder {idx}/{len(subfolders)}: {os.path.basename(subfolder)} ===")
            process_folder(subfolder)

        print("\nBatch processing completed for all subfolders!")

🛑 Error: GEMINI_API_KEY environment variable not set.

Starting batch processing for: /content/main_folder
Found 21 subfolder(s) to process

=== Processing subfolder 1/21: .ipynb_checkpoints ===

Starting OCR process for folder: '.ipynb_checkpoints'
  Subject detected: '.ipynb_checkpoints'
  Output CSV will be: '/content/main_folder/.ipynb_checkpoints.csv'
  No PNG files found.

=== Processing subfolder 2/21: Commentaire de Diderot (AGREG ext. - note _ 10) ===

Starting OCR process for folder: 'Commentaire de Diderot (AGREG ext. - note _ 10)'
  Subject detected: 'Commentaire de Diderot (AGREG ext. - note _ 10)'
  Output CSV will be: '/content/main_folder/Commentaire de Diderot (AGREG ext. - note _ 10).csv'
  Found 42 chunks to process.


Processing Chunks:   0%|          | 0/42 [00:00<?, ?it/s]


Completed processing for: Commentaire de Diderot (AGREG ext. - note _ 10)
CSV saved at: /content/main_folder/Commentaire de Diderot (AGREG ext. - note _ 10).csv

=== Processing subfolder 3/21: Commentaire de Diderot (CAPES 2019 - note _ 19) ===

Starting OCR process for folder: 'Commentaire de Diderot (CAPES 2019 - note _ 19)'
  Subject detected: 'Commentaire de Diderot (CAPES 2019 - note _ 19)'
  Output CSV will be: '/content/main_folder/Commentaire de Diderot (CAPES 2019 - note _ 19).csv'
  Found 34 chunks to process.


Processing Chunks:   0%|          | 0/34 [00:00<?, ?it/s]


Completed processing for: Commentaire de Diderot (CAPES 2019 - note _ 19)
CSV saved at: /content/main_folder/Commentaire de Diderot (CAPES 2019 - note _ 19).csv

=== Processing subfolder 4/21: Commentaire de Foucault - Naissance de la clinique (AGREG interne 2021 - note 13) ===

Starting OCR process for folder: 'Commentaire de Foucault - Naissance de la clinique (AGREG interne 2021 - note 13)'
  Subject detected: 'Commentaire de Foucault - Naissance de la clinique (AGREG interne 2021 - note 13)'
  Output CSV will be: '/content/main_folder/Commentaire de Foucault - Naissance de la clinique (AGREG interne 2021 - note 13).csv'
  Found 39 chunks to process.


Processing Chunks:   0%|          | 0/39 [00:00<?, ?it/s]

In [ ]:
#@title CSV File mover

import os
import shutil
import glob

def move_csv_files(source_dir, target_dir):
    # Create the target directory if it doesn't exist
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
        print(f"Created directory: {target_dir}")

    # Find all CSV files in the source directory
    csv_pattern = os.path.join(source_dir, "*.csv")
    csv_files = glob.glob(csv_pattern)

    # Also search in subdirectories
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            if file.lower().endswith('.csv'):
                file_path = os.path.join(root, file)
                if file_path not in csv_files:  # Avoid duplicates
                    csv_files.append(file_path)

    # Move each CSV file to the target directory
    moved_count = 0
    for file_path in csv_files:
        file_name = os.path.basename(file_path)
        destination = os.path.join(target_dir, file_name)

        # Handle file name conflicts
        if os.path.exists(destination):
            base, extension = os.path.splitext(file_name)
            counter = 1
            while os.path.exists(destination):
                new_name = f"{base}_{counter}{extension}"
                destination = os.path.join(target_dir, new_name)
                counter += 1

        # Move the file
        shutil.move(file_path, destination)
        moved_count += 1
        print(f"Moved: {file_path} → {destination}")

    return moved_count

if __name__ == "__main__":
    source_directory = "/content/main_folder"
    target_directory = "/content/CSV"

    print(f"Searching for CSV files in {source_directory}...")
    moved = move_csv_files(source_directory, target_directory)
    print(f"Successfully moved {moved} CSV file(s) to {target_directory}")

In [ ]:
import shutil
import os

src = '/content/drive/MyDrive/folder_partitioned_merged/Le bonheur est-il affaire de hasard ou de nécessité ? (AGREG interne 2022 - note : 14)'
dst = '/content/Le bonheur est-il affaire de hasard ou de nécessité 14 ? (AGREG interne 2022 - note : 14)'

if os.path.exists(dst):
    shutil.rmtree(dst)

shutil.copytree(src, dst)

In [ ]:
#@title OCR avec enrengistrement en CSV au même nom

import os
import google.generativeai as genai
import csv
import time
import re # For natural sorting and filename sanitization
from tqdm.notebook import tqdm # Progress bars
import traceback # For detailed error printing

# --- Configuration ---
TARGET_FOLDER_PATH = "/content/folder"
# Output CSV file path will be generated dynamically later

try:
    genai.configure(api_key=os.environ["GEMINI_API_KEY"])
    print("Gemini API Key configured.")
except KeyError:
    print("🛑 Error: GEMINI_API_KEY environment variable not set.")
    # sys.exit(1) # Optional: exit if running unattended
except Exception as e:
    print(f"🛑 Error configuring Gemini API: {e}")
    # sys.exit(1) # Optional: exit if running unattended


# Gemini Model Configuration)
GENERATION_CONFIG = {
  "temperature": 0.2,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}
MODEL_NAME = "gemini-2.5-pro-preview-03-25"


# System Prompts (remain the same)
SYSTEM_INSTRUCTION_WITH_CONTEXT = """Tâche Principale : Recopie aussi fidèlement que possible le texte manuscrit de l'image fournie. Répond **exclusivement** avec le texte transcrit, sans aucune introduction, commentaire, ou note.

Contexte de la Dissertation : Ce texte est un extrait d'une dissertation de philosophie portant sur le sujet "{sujet_copie}".

Mise en forme :
- Si des mots ou groupes de mots sont soulignés dans l'image, mets-les en italique en markdown (*texte souligné*).
- Si un mot est barré, encadre-le de tildes en markdown (~mot barré~).

Aide Contextuelle (Texte Précédent) : Pour t'aider à déchiffrer l'écriture, le texte qui précède *immédiatement* celui de l'image actuelle est fourni ci-dessous (séparé par '--- Texte Précédent ---'). Ce texte a été obtenu par OCR des sections précédentes de la dissertation. Le texte de l'image actuelle commence là où le texte précédent s'arrête (il peut y avoir un léger chevauchement de lignes dû au découpage des images).

**Priorité :** Ta tâche principale reste de transcrire *exactement* ce que tu vois sur l'image actuelle. Utilise le texte précédent **uniquement** comme une aide *secondaire* pour résoudre des ambiguïtés ou déchiffrer des mots très difficiles. **Ne laisse PAS le texte précédent remplacer ou modifier ce qui est clairement visible sur l'image actuelle.** En cas de conflit entre le texte précédent et l'image, **la transcription fidèle de l'image PRÉVAUT.**
"""

SYSTEM_INSTRUCTION_FIRST_CHUNK = """Tâche Principale : Recopie aussi fidèlement que possible le texte manuscrit de l'image fournie. Répond **exclusivement** avec le texte transcrit, sans aucune introduction, commentaire, ou note.

Contexte de la Dissertation : Ce texte est un extrait d'une dissertation de philosophie portant sur le sujet "{sujet_copie}". Ceci est le tout début de la dissertation.

Mise en forme :
- Si des mots ou groupes de mots sont soulignés dans l'image, mets-les en italique en markdown (*texte souligné*).
- Si un mot est barré, encadre-le de tildes en markdown (~mot barré~).
"""

# --- Helper Functions ---

def extract_folder_info(folder_path):
    """Extracts the subject and full name from a folder path name."""
    if not os.path.isdir(folder_path): return None, None
    full_name = os.path.basename(folder_path)
    subject = None
    index_parenthese = full_name.find('(')
    if index_parenthese != -1: subject = full_name[:index_parenthese].strip()
    else: subject = full_name.strip() # Fallback
    return subject, full_name

# --- NEW: Function to sanitize filename ---
def sanitize_filename(filename):
    """Removes or replaces characters invalid for filenames."""
    # Remove leading/trailing whitespace
    sanitized = filename.strip()
    # Replace common invalid characters with underscore (adjust as needed)
    sanitized = re.sub(r'[\\/*?:"<>|]', '_', sanitized)
    # Replace forward slash specifically (common in paths)
    sanitized = sanitized.replace('/', '_')
    # Optional: Limit length if necessary (e.g., 200 chars)
    # max_len = 200
    # if len(sanitized) > max_len:
    #     name, ext = os.path.splitext(sanitized)
    #     sanitized = name[:max_len - len(ext) - 1] + '~' + ext
    return sanitized


# ... (upload_to_gemini, delete_gemini_file, natural_sort_key remain the same) ...
def upload_to_gemini(path, mime_type="image/png"):
    """Uploads a file to Gemini and returns the File object."""
    try:
        file = genai.upload_file(path, mime_type=mime_type)
        return file
    except Exception as e:
        print(f"  🛑 Error uploading file {path}: {e}")
        return None

def delete_gemini_file(file_object):
    """Deletes a file from Gemini service."""
    if file_object:
        try: genai.delete_file(file_object.name)
        except Exception as e: print(f"  ⚠️ Warning: Failed to delete file {file_object.name}: {e}")

def natural_sort_key(s):
    """Key for sorting strings containing numbers naturally."""
    return [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', s)]


# --- Main Processing Logic for Single Folder ---

if __name__ == "__main__":
    if not os.path.isdir(TARGET_FOLDER_PATH):
        print(f"🛑 Error: Target folder not found at '{TARGET_FOLDER_PATH}'")
    else:
        print(f"Starting OCR process for single folder: '{os.path.basename(TARGET_FOLDER_PATH)}'")

        # 1. Extract Subject and Full Name for the target folder
        sujet_copie, nom_complet_sous_dossier = extract_folder_info(TARGET_FOLDER_PATH)

        if sujet_copie is None:
            print(f"🛑 Error: Could not extract info from folder path '{TARGET_FOLDER_PATH}'. Cannot proceed.")
        else:
            print(f"  Subject detected: '{sujet_copie}'")

            # --- GENERATE DYNAMIC OUTPUT CSV FILENAME ---
            parent_dir = os.path.dirname(TARGET_FOLDER_PATH) # Get directory containing the target folder
            sanitized_subject = sanitize_filename(sujet_copie) # Sanitize the subject for use in filename
            OUTPUT_CSV_FILE = os.path.join(parent_dir, f"{sanitized_subject}.csv")
            print(f"  Output CSV will be saved as: '{OUTPUT_CSV_FILE}'")
            # --- End Filename Generation ---

            # --- Initialize CSV Writer ---
            try:
                with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as csvfile:
                    writer = csv.writer(csvfile)
                    print("  CSV file opened successfully.")

                    # Write the folder name (title) to the CSV
                    writer.writerow([nom_complet_sous_dossier])
                    print(f"  Wrote folder title to CSV: {nom_complet_sous_dossier}")

                    # 2. Find and Sort Image Chunks
                    # ... (Chunk finding logic remains the same) ...
                    try:
                        all_files = [f for f in os.listdir(TARGET_FOLDER_PATH) if f.lower().endswith(".png")]
                        sorted_chunks = sorted(all_files, key=natural_sort_key)
                    except OSError as e:
                        print(f"  🛑 Error listing files in {TARGET_FOLDER_PATH}: {e}")
                        sorted_chunks = []


                    if not sorted_chunks:
                        print("  No PNG chunk files found in this folder.")
                    else:
                        print(f"  Found {len(sorted_chunks)} chunk(s) to process.")

                        # --- Variables to store previous OCR results ---
                        ocr_result_n_minus_1 = ""
                        ocr_result_n_minus_2 = ""

                        # --- Loop through Image Chunks ---
                        # ... (The rest of the loop logic remains exactly the same as the previous version) ...
                        for i, chunk_filename in enumerate(tqdm(sorted_chunks, desc="Processing Chunks")):
                            chunk_path = os.path.join(TARGET_FOLDER_PATH, chunk_filename)
                            gemini_file = None
                            current_ocr_result = "" # Initialize for this chunk

                            print(f"\n    Processing chunk {i+1}/{len(sorted_chunks)}: {chunk_filename}")

                            try:
                                # 3. Select System Prompt and Prepare Context
                                prompt_parts = []
                                if i == 0:
                                    # First chunk: No preceding context
                                    current_system_instruction = SYSTEM_INSTRUCTION_FIRST_CHUNK.format(sujet_copie=sujet_copie)
                                    print("      Using FIRST chunk system prompt.")
                                else:
                                    # Subsequent chunks: Prepare context text
                                    current_system_instruction = SYSTEM_INSTRUCTION_WITH_CONTEXT.format(sujet_copie=sujet_copie)
                                    context_text = "\n--- Texte Précédent ---\n"
                                    if ocr_result_n_minus_2: # Add N-2 if available
                                        context_text += ocr_result_n_minus_2 + "\n" # Add newline separator
                                    if ocr_result_n_minus_1: # Add N-1 if available
                                        context_text += ocr_result_n_minus_1
                                    context_text += "\n--- Fin du Texte Précédent ---\n\n--- Image Actuelle à Transcrire ---"
                                    # Add context text as the first part of the prompt
                                    prompt_parts.append(context_text)
                                    print(f"      Using CONTEXT system prompt (Context length: ~{len(context_text)} chars).")


                                # 4. Upload Image Chunk
                                print(f"      Uploading: {chunk_filename}...")
                                gemini_file = upload_to_gemini(chunk_path)
                                if gemini_file is None:
                                    current_ocr_result = f"ERROR: Upload failed for {chunk_filename}"
                                    writer.writerow([current_ocr_result])
                                    print(f"      {current_ocr_result}")
                                    continue # Skip to next chunk

                                # Add the image file as the next part of the prompt
                                prompt_parts.append(gemini_file)

                                # 5. Initialize Model (Can potentially be outside loop if system prompt doesn't change often, but safer inside for now)
                                # Re-create model instance ensures the correct system prompt is used
                                model = genai.GenerativeModel(
                                    model_name=MODEL_NAME,
                                    generation_config=GENERATION_CONFIG,
                                    system_instruction=current_system_instruction,
                                )

                                # 6. Call Gemini API
                                print("      Calling Gemini API...")
                                response = model.generate_content(prompt_parts, request_options={'timeout': 600}) # Increased timeout

                                # 7. Extract OCR Result
                                if response and response.parts:
                                     try:
                                         current_ocr_result = response.text
                                         # Optional: Remove potential leading/trailing whitespace
                                         current_ocr_result = current_ocr_result.strip()
                                         print(f"      OCR result received (~{len(current_ocr_result)} chars).")
                                     except ValueError:
                                         print(f"      ⚠️ Warning: Could not extract text from response for {chunk_filename}. Response: {response.prompt_feedback}")
                                         current_ocr_result = f"ERROR: No text found or content blocked ({response.prompt_feedback})"
                                     except Exception as e_resp:
                                        print(f"      🛑 Error extracting text from response: {e_resp}")
                                        current_ocr_result = f"ERROR: Response parsing failed - {e_resp}"
                                else:
                                    print(f"      ⚠️ Warning: Empty or invalid response received for {chunk_filename}.")
                                    current_ocr_result = "ERROR: Empty or invalid response from API"

                                # Write result to CSV
                                writer.writerow([current_ocr_result])
                                # print(f"      Wrote OCR result to CSV.")

                                # 8. Update History for Next Iteration
                                ocr_result_n_minus_2 = ocr_result_n_minus_1
                                ocr_result_n_minus_1 = current_ocr_result # Store the raw result from this chunk

                            except Exception as e_api:
                                print(f"    🛑 An error occurred during API call or processing for {chunk_filename}:")
                                traceback.print_exc() # Print full traceback for debugging
                                error_message = f"ERROR: API call failed for {chunk_filename} - {type(e_api).__name__}"
                                writer.writerow([error_message])
                                # Update history with error marker to avoid propagating bad context
                                ocr_result_n_minus_2 = ocr_result_n_minus_1
                                ocr_result_n_minus_1 = error_message # Store error marker

                            finally:
                                # 9. Delete Uploaded File from Gemini
                                if gemini_file:
                                    delete_gemini_file(gemini_file)
                                time.sleep(1.5) # Slightly increased delay


            except IOError as e:
                print(f"🛑 Error opening or writing to CSV file {OUTPUT_CSV_FILE}: {e}")
            except Exception as e_main:
                 print(f"🛑 An unexpected error occurred during the main process:")
                 traceback.print_exc()

            print("\n--- OCR Process Complete for Folder ---")
            print(f"Results saved in: {OUTPUT_CSV_FILE}") # Display the final generated path

Gemini API Key configured.
Starting OCR process for single folder: 'Le bonheur est-il affaire de hasard et de nécessité 15.5 ? (AGREG interne -  note : 15,5)'
  Subject detected: 'Le bonheur est-il affaire de hasard et de nécessité 15.5 ?'
  Output CSV will be saved as: '/content/Le bonheur est-il affaire de hasard et de nécessité 15.5 _.csv'
  CSV file opened successfully.
  Wrote folder title to CSV: Le bonheur est-il affaire de hasard et de nécessité 15.5 ? (AGREG interne -  note : 15,5)
  Found 51 chunk(s) to process.


Processing Chunks:   0%|          | 0/51 [00:00<?, ?it/s]


    Processing chunk 1/51: 0001.png
      Using FIRST chunk system prompt.
      Uploading: 0001.png...
      Calling Gemini API...
      OCR result received (~719 chars).
  ⚠️ Warning: Failed to delete file files/ars31p8iz8mm: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

    Processing chunk 2/51: 0002.png
      Using CONTEXT system prompt (Context length: ~812 chars).
      Uploading: 0002.png...
      Calling Gemini API...
      OCR result received (~651 chars).
  ⚠️ Warning: Failed to delete file files/7zacz5knf2rr: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

    Processing chunk 3/51: 0003.png
      Using CONTEXT system prompt (Context length: ~1464 chars).
      Uploading: 0003.png...
      Calling Gemini API...
      OCR result received (~311 chars).

    Processing chunk 4/51: 0004.png
      Using CONTEXT system prompt (Context length: ~1056 chars).
      Uploading: 0004.png...
      Calling Gemini API...
 

In [ ]:
#@title OCR avec enrengistrement en CSV au même nom

import os
import google.generativeai as genai
import csv
import time
import re # For natural sorting and filename sanitization
from tqdm.notebook import tqdm # Progress bars
import traceback # For detailed error printing

# --- Configuration ---
# *** SET THE TARGET FOLDER HERE ***
TARGET_FOLDER_PATH = "/content/Peut-on vivre en paix avec son inconscient ? (AGREG interne 2021 -note : 12)" # Example: Set to the specific subfolder path you want to process

# Output CSV file path will be generated dynamically later

# Gemini API Key (ensure it's set as an environment variable)
# ... (API Key configuration remains the same) ...
try:
    genai.configure(api_key=os.environ["GEMINI_API_KEY"])
    print("Gemini API Key configured.")
except KeyError:
    print("🛑 Error: GEMINI_API_KEY environment variable not set.")
    # sys.exit(1) # Optional: exit if running unattended
except Exception as e:
    print(f"🛑 Error configuring Gemini API: {e}")
    # sys.exit(1) # Optional: exit if running unattended


# Gemini Model Configuration)
GENERATION_CONFIG = {
  "temperature": 0.2, # Slightly lower temp might encourage more fidelity
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}
MODEL_NAME = "gemini-2.5-pro-preview-03-25"


# System Prompts (remain the same)
SYSTEM_INSTRUCTION_WITH_CONTEXT = """Tâche Principale : Recopie aussi fidèlement que possible le texte manuscrit de l'image fournie. Répond **exclusivement** avec le texte transcrit, sans aucune introduction, commentaire, ou note.

Contexte de la Dissertation : Ce texte est un extrait d'une dissertation de philosophie portant sur le sujet "{sujet_copie}".

Mise en forme :
- Si des mots ou groupes de mots sont soulignés dans l'image, mets-les en italique en markdown (*texte souligné*).
- Si un mot est barré, encadre-le de tildes en markdown (~mot barré~).

Aide Contextuelle (Texte Précédent) : Pour t'aider à déchiffrer l'écriture, le texte qui précède *immédiatement* celui de l'image actuelle est fourni ci-dessous (séparé par '--- Texte Précédent ---'). Ce texte a été obtenu par OCR des sections précédentes de la dissertation. Le texte de l'image actuelle commence là où le texte précédent s'arrête (il peut y avoir un léger chevauchement de lignes dû au découpage des images).

**Priorité Absolue :** Ta tâche principale reste de transcrire *exactement* ce que tu vois sur l'image actuelle. Utilise le texte précédent **uniquement** comme une aide *secondaire* pour résoudre des ambiguïtés ou déchiffrer des mots très difficiles. **Ne laisse PAS le texte précédent remplacer ou modifier ce qui est clairement visible sur l'image actuelle.** En cas de conflit entre le texte précédent et l'image, **la transcription fidèle de l'image PRÉVAUT.**
"""

SYSTEM_INSTRUCTION_FIRST_CHUNK = """Tâche Principale : Recopie aussi fidèlement que possible le texte manuscrit de l'image fournie. Répond **exclusivement** avec le texte transcrit, sans aucune introduction, commentaire, ou note.

Contexte de la Dissertation : Ce texte est un extrait d'une dissertation de philosophie portant sur le sujet "{sujet_copie}". Ceci est le tout début de la dissertation.

Mise en forme :
- Si des mots ou groupes de mots sont soulignés dans l'image, mets-les en italique en markdown (*texte souligné*).
- Si un mot est barré, encadre-le de tildes en markdown (~mot barré~).
"""

# --- Helper Functions ---

def extract_folder_info(folder_path):
    """Extracts the subject and full name from a folder path name."""
    if not os.path.isdir(folder_path): return None, None
    full_name = os.path.basename(folder_path)
    subject = None
    index_parenthese = full_name.find('(')
    if index_parenthese != -1: subject = full_name[:index_parenthese].strip()
    else: subject = full_name.strip() # Fallback
    return subject, full_name

# --- NEW: Function to sanitize filename ---
def sanitize_filename(filename):
    """Removes or replaces characters invalid for filenames."""
    # Remove leading/trailing whitespace
    sanitized = filename.strip()
    # Replace common invalid characters with underscore (adjust as needed)
    sanitized = re.sub(r'[\\/*?:"<>|]', '_', sanitized)
    # Replace forward slash specifically (common in paths)
    sanitized = sanitized.replace('/', '_')
    # Optional: Limit length if necessary (e.g., 200 chars)
    # max_len = 200
    # if len(sanitized) > max_len:
    #     name, ext = os.path.splitext(sanitized)
    #     sanitized = name[:max_len - len(ext) - 1] + '~' + ext
    return sanitized


# ... (upload_to_gemini, delete_gemini_file, natural_sort_key remain the same) ...
def upload_to_gemini(path, mime_type="image/png"):
    """Uploads a file to Gemini and returns the File object."""
    try:
        file = genai.upload_file(path, mime_type=mime_type)
        return file
    except Exception as e:
        print(f"  🛑 Error uploading file {path}: {e}")
        return None

def delete_gemini_file(file_object):
    """Deletes a file from Gemini service."""
    if file_object:
        try: genai.delete_file(file_object.name)
        except Exception as e: print(f"  ⚠️ Warning: Failed to delete file {file_object.name}: {e}")

def natural_sort_key(s):
    """Key for sorting strings containing numbers naturally."""
    return [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', s)]


# --- Main Processing Logic for Single Folder ---

if __name__ == "__main__":
    if not os.path.isdir(TARGET_FOLDER_PATH):
        print(f"🛑 Error: Target folder not found at '{TARGET_FOLDER_PATH}'")
    else:
        print(f"Starting OCR process for single folder: '{os.path.basename(TARGET_FOLDER_PATH)}'")

        # 1. Extract Subject and Full Name for the target folder
        sujet_copie, nom_complet_sous_dossier = extract_folder_info(TARGET_FOLDER_PATH)

        if sujet_copie is None:
            print(f"🛑 Error: Could not extract info from folder path '{TARGET_FOLDER_PATH}'. Cannot proceed.")
        else:
            print(f"  Subject detected: '{sujet_copie}'")

            # --- GENERATE DYNAMIC OUTPUT CSV FILENAME ---
            parent_dir = os.path.dirname(TARGET_FOLDER_PATH) # Get directory containing the target folder
            sanitized_subject = sanitize_filename(sujet_copie) # Sanitize the subject for use in filename
            OUTPUT_CSV_FILE = os.path.join(parent_dir, f"{sanitized_subject}.csv")
            print(f"  Output CSV will be saved as: '{OUTPUT_CSV_FILE}'")
            # --- End Filename Generation ---

            # --- Initialize CSV Writer ---
            try:
                with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as csvfile:
                    writer = csv.writer(csvfile)
                    print("  CSV file opened successfully.")

                    # Write the folder name (title) to the CSV
                    writer.writerow([nom_complet_sous_dossier])
                    print(f"  Wrote folder title to CSV: {nom_complet_sous_dossier}")

                    # 2. Find and Sort Image Chunks
                    # ... (Chunk finding logic remains the same) ...
                    try:
                        all_files = [f for f in os.listdir(TARGET_FOLDER_PATH) if f.lower().endswith(".png")]
                        sorted_chunks = sorted(all_files, key=natural_sort_key)
                    except OSError as e:
                        print(f"  🛑 Error listing files in {TARGET_FOLDER_PATH}: {e}")
                        sorted_chunks = []


                    if not sorted_chunks:
                        print("  No PNG chunk files found in this folder.")
                    else:
                        print(f"  Found {len(sorted_chunks)} chunk(s) to process.")

                        # --- Variables to store previous OCR results ---
                        ocr_result_n_minus_1 = ""
                        ocr_result_n_minus_2 = ""

                        # --- Loop through Image Chunks ---
                        # ... (The rest of the loop logic remains exactly the same as the previous version) ...
                        for i, chunk_filename in enumerate(tqdm(sorted_chunks, desc="Processing Chunks")):
                            chunk_path = os.path.join(TARGET_FOLDER_PATH, chunk_filename)
                            gemini_file = None
                            current_ocr_result = "" # Initialize for this chunk

                            print(f"\n    Processing chunk {i+1}/{len(sorted_chunks)}: {chunk_filename}")

                            try:
                                # 3. Select System Prompt and Prepare Context
                                prompt_parts = []
                                if i == 0:
                                    # First chunk: No preceding context
                                    current_system_instruction = SYSTEM_INSTRUCTION_FIRST_CHUNK.format(sujet_copie=sujet_copie)
                                    print("      Using FIRST chunk system prompt.")
                                else:
                                    # Subsequent chunks: Prepare context text
                                    current_system_instruction = SYSTEM_INSTRUCTION_WITH_CONTEXT.format(sujet_copie=sujet_copie)
                                    context_text = "\n--- Texte Précédent ---\n"
                                    if ocr_result_n_minus_2: # Add N-2 if available
                                        context_text += ocr_result_n_minus_2 + "\n" # Add newline separator
                                    if ocr_result_n_minus_1: # Add N-1 if available
                                        context_text += ocr_result_n_minus_1
                                    context_text += "\n--- Fin du Texte Précédent ---\n\n--- Image Actuelle à Transcrire ---"
                                    # Add context text as the first part of the prompt
                                    prompt_parts.append(context_text)
                                    print(f"      Using CONTEXT system prompt (Context length: ~{len(context_text)} chars).")


                                # 4. Upload Image Chunk
                                print(f"      Uploading: {chunk_filename}...")
                                gemini_file = upload_to_gemini(chunk_path)
                                if gemini_file is None:
                                    current_ocr_result = f"ERROR: Upload failed for {chunk_filename}"
                                    writer.writerow([current_ocr_result])
                                    print(f"      {current_ocr_result}")
                                    continue # Skip to next chunk

                                # Add the image file as the next part of the prompt
                                prompt_parts.append(gemini_file)

                                # 5. Initialize Model (Can potentially be outside loop if system prompt doesn't change often, but safer inside for now)
                                # Re-create model instance ensures the correct system prompt is used
                                model = genai.GenerativeModel(
                                    model_name=MODEL_NAME,
                                    generation_config=GENERATION_CONFIG,
                                    system_instruction=current_system_instruction,
                                )

                                # 6. Call Gemini API
                                print("      Calling Gemini API...")
                                response = model.generate_content(prompt_parts, request_options={'timeout': 600}) # Increased timeout

                                # 7. Extract OCR Result
                                if response and response.parts:
                                     try:
                                         current_ocr_result = response.text
                                         # Optional: Remove potential leading/trailing whitespace
                                         current_ocr_result = current_ocr_result.strip()
                                         print(f"      OCR result received (~{len(current_ocr_result)} chars).")
                                     except ValueError:
                                         print(f"      ⚠️ Warning: Could not extract text from response for {chunk_filename}. Response: {response.prompt_feedback}")
                                         current_ocr_result = f"ERROR: No text found or content blocked ({response.prompt_feedback})"
                                     except Exception as e_resp:
                                        print(f"      🛑 Error extracting text from response: {e_resp}")
                                        current_ocr_result = f"ERROR: Response parsing failed - {e_resp}"
                                else:
                                    print(f"      ⚠️ Warning: Empty or invalid response received for {chunk_filename}.")
                                    current_ocr_result = "ERROR: Empty or invalid response from API"

                                # Write result to CSV
                                writer.writerow([current_ocr_result])
                                # print(f"      Wrote OCR result to CSV.")

                                # 8. Update History for Next Iteration
                                ocr_result_n_minus_2 = ocr_result_n_minus_1
                                ocr_result_n_minus_1 = current_ocr_result # Store the raw result from this chunk

                            except Exception as e_api:
                                print(f"    🛑 An error occurred during API call or processing for {chunk_filename}:")
                                traceback.print_exc() # Print full traceback for debugging
                                error_message = f"ERROR: API call failed for {chunk_filename} - {type(e_api).__name__}"
                                writer.writerow([error_message])
                                # Update history with error marker to avoid propagating bad context
                                ocr_result_n_minus_2 = ocr_result_n_minus_1
                                ocr_result_n_minus_1 = error_message # Store error marker

                            finally:
                                # 9. Delete Uploaded File from Gemini
                                if gemini_file:
                                    delete_gemini_file(gemini_file)
                                time.sleep(1.5) # Slightly increased delay


            except IOError as e:
                print(f"🛑 Error opening or writing to CSV file {OUTPUT_CSV_FILE}: {e}")
            except Exception as e_main:
                 print(f"🛑 An unexpected error occurred during the main process:")
                 traceback.print_exc()

            print("\n--- OCR Process Complete for Folder ---")
            print(f"Results saved in: {OUTPUT_CSV_FILE}") # Display the final generated path

Gemini API Key configured.
Starting OCR process for single folder: 'Peut-on vivre en paix avec son inconscient ? (AGREG interne 2021 -note : 12)'
  Subject detected: 'Peut-on vivre en paix avec son inconscient ?'
  Output CSV will be saved as: '/content/Peut-on vivre en paix avec son inconscient _.csv'
  CSV file opened successfully.
  Wrote folder title to CSV: Peut-on vivre en paix avec son inconscient ? (AGREG interne 2021 -note : 12)
  Found 45 chunk(s) to process.


Processing Chunks:   0%|          | 0/45 [00:00<?, ?it/s]


    Processing chunk 1/45: 0001.png
      Using FIRST chunk system prompt.
      Uploading: 0001.png...
      Calling Gemini API...
      OCR result received (~668 chars).

    Processing chunk 2/45: 0002.png
      Using CONTEXT system prompt (Context length: ~761 chars).
      Uploading: 0002.png...
      Calling Gemini API...
      OCR result received (~641 chars).

    Processing chunk 3/45: 0003.png
      Using CONTEXT system prompt (Context length: ~1403 chars).
      Uploading: 0003.png...
      Calling Gemini API...
      OCR result received (~328 chars).

    Processing chunk 4/45: 0004.png
      Using CONTEXT system prompt (Context length: ~1063 chars).
      Uploading: 0004.png...
      Calling Gemini API...
      OCR result received (~645 chars).

    Processing chunk 5/45: 0005.png
      Using CONTEXT system prompt (Context length: ~1067 chars).
      Uploading: 0005.png...
      Calling Gemini API...
      OCR result received (~631 chars).

    Processing chunk 6/45: 0006

In [ ]:
import shutil
import os

src = '/content/drive/MyDrive/folder_partitioned_merged/Le bonheur est-il affaire de hasard ou de nécessité ? (AGREG interne 2022 - note : 8)'
dst = '/content/Le bonheur est-il affaire de hasard ou de nécessité ? (AGREG interne 2022 - note : 8)'

if os.path.exists(dst):
    shutil.rmtree(dst)

shutil.copytree(src, dst)

'/content/Le bonheur est-il affaire de hasard ou de nécessité ? (AGREG interne 2022 - note : 8)'

In [ ]:
#@title OCR avec enrengistrement en CSV au même nom

import os
import google.generativeai as genai
import csv
import time
import re # For natural sorting and filename sanitization
from tqdm.notebook import tqdm # Progress bars
import traceback # For detailed error printing

# --- Configuration ---
# *** SET THE TARGET FOLDER HERE ***
TARGET_FOLDER_PATH = "/content/Le bonheur est-il affaire de hasard ou de nécessité ? (AGREG interne 2022 - note : 8)" # Example: Set to the specific subfolder path you want to process

# Output CSV file path will be generated dynamically later

# Gemini API Key (ensure it's set as an environment variable)
# ... (API Key configuration remains the same) ...
try:
    genai.configure(api_key=os.environ["GEMINI_API_KEY"])
    print("Gemini API Key configured.")
except KeyError:
    print("🛑 Error: GEMINI_API_KEY environment variable not set.")
    # sys.exit(1) # Optional: exit if running unattended
except Exception as e:
    print(f"🛑 Error configuring Gemini API: {e}")
    # sys.exit(1) # Optional: exit if running unattended


# Gemini Model Configuration)
GENERATION_CONFIG = {
  "temperature": 0.2, # Slightly lower temp might encourage more fidelity
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}
MODEL_NAME = "gemini-2.5-pro-preview-03-25"


# System Prompts (remain the same)
SYSTEM_INSTRUCTION_WITH_CONTEXT = """Tâche Principale : Recopie aussi fidèlement que possible le texte manuscrit de l'image fournie. Répond **exclusivement** avec le texte transcrit, sans aucune introduction, commentaire, ou note.

Contexte de la Dissertation : Ce texte est un extrait d'une dissertation de philosophie portant sur le sujet "{sujet_copie}".

Mise en forme :
- Si des mots ou groupes de mots sont soulignés dans l'image, mets-les en italique en markdown (*texte souligné*).
- Si un mot est barré, encadre-le de tildes en markdown (~mot barré~).

Aide Contextuelle (Texte Précédent) : Pour t'aider à déchiffrer l'écriture, le texte qui précède *immédiatement* celui de l'image actuelle est fourni ci-dessous (séparé par '--- Texte Précédent ---'). Ce texte a été obtenu par OCR des sections précédentes de la dissertation. Le texte de l'image actuelle commence là où le texte précédent s'arrête (il peut y avoir un léger chevauchement de lignes dû au découpage des images).

**Priorité Absolue :** Ta tâche principale reste de transcrire *exactement* ce que tu vois sur l'image actuelle. Utilise le texte précédent **uniquement** comme une aide *secondaire* pour résoudre des ambiguïtés ou déchiffrer des mots très difficiles. **Ne laisse PAS le texte précédent remplacer ou modifier ce qui est clairement visible sur l'image actuelle.** En cas de conflit entre le texte précédent et l'image, **la transcription fidèle de l'image PRÉVAUT.**
"""

SYSTEM_INSTRUCTION_FIRST_CHUNK = """Tâche Principale : Recopie aussi fidèlement que possible le texte manuscrit de l'image fournie. Répond **exclusivement** avec le texte transcrit, sans aucune introduction, commentaire, ou note.

Contexte de la Dissertation : Ce texte est un extrait d'une dissertation de philosophie portant sur le sujet "{sujet_copie}". Ceci est le tout début de la dissertation.

Mise en forme :
- Si des mots ou groupes de mots sont soulignés dans l'image, mets-les en italique en markdown (*texte souligné*).
- Si un mot est barré, encadre-le de tildes en markdown (~mot barré~).
"""

# --- Helper Functions ---

def extract_folder_info(folder_path):
    """Extracts the subject and full name from a folder path name."""
    if not os.path.isdir(folder_path): return None, None
    full_name = os.path.basename(folder_path)
    subject = None
    index_parenthese = full_name.find('(')
    if index_parenthese != -1: subject = full_name[:index_parenthese].strip()
    else: subject = full_name.strip() # Fallback
    return subject, full_name

# --- NEW: Function to sanitize filename ---
def sanitize_filename(filename):
    """Removes or replaces characters invalid for filenames."""
    # Remove leading/trailing whitespace
    sanitized = filename.strip()
    # Replace common invalid characters with underscore (adjust as needed)
    sanitized = re.sub(r'[\\/*?:"<>|]', '_', sanitized)
    # Replace forward slash specifically (common in paths)
    sanitized = sanitized.replace('/', '_')
    # Optional: Limit length if necessary (e.g., 200 chars)
    # max_len = 200
    # if len(sanitized) > max_len:
    #     name, ext = os.path.splitext(sanitized)
    #     sanitized = name[:max_len - len(ext) - 1] + '~' + ext
    return sanitized


# ... (upload_to_gemini, delete_gemini_file, natural_sort_key remain the same) ...
def upload_to_gemini(path, mime_type="image/png"):
    """Uploads a file to Gemini and returns the File object."""
    try:
        file = genai.upload_file(path, mime_type=mime_type)
        return file
    except Exception as e:
        print(f"  🛑 Error uploading file {path}: {e}")
        return None

def delete_gemini_file(file_object):
    """Deletes a file from Gemini service."""
    if file_object:
        try: genai.delete_file(file_object.name)
        except Exception as e: print(f"  ⚠️ Warning: Failed to delete file {file_object.name}: {e}")

def natural_sort_key(s):
    """Key for sorting strings containing numbers naturally."""
    return [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', s)]


# --- Main Processing Logic for Single Folder ---

if __name__ == "__main__":
    if not os.path.isdir(TARGET_FOLDER_PATH):
        print(f"🛑 Error: Target folder not found at '{TARGET_FOLDER_PATH}'")
    else:
        print(f"Starting OCR process for single folder: '{os.path.basename(TARGET_FOLDER_PATH)}'")

        # 1. Extract Subject and Full Name for the target folder
        sujet_copie, nom_complet_sous_dossier = extract_folder_info(TARGET_FOLDER_PATH)

        if sujet_copie is None:
            print(f"🛑 Error: Could not extract info from folder path '{TARGET_FOLDER_PATH}'. Cannot proceed.")
        else:
            print(f"  Subject detected: '{sujet_copie}'")

            # --- GENERATE DYNAMIC OUTPUT CSV FILENAME ---
            parent_dir = os.path.dirname(TARGET_FOLDER_PATH) # Get directory containing the target folder
            sanitized_subject = sanitize_filename(sujet_copie) # Sanitize the subject for use in filename
            OUTPUT_CSV_FILE = os.path.join(parent_dir, f"{sanitized_subject}.csv")
            print(f"  Output CSV will be saved as: '{OUTPUT_CSV_FILE}'")
            # --- End Filename Generation ---

            # --- Initialize CSV Writer ---
            try:
                with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as csvfile:
                    writer = csv.writer(csvfile)
                    print("  CSV file opened successfully.")

                    # Write the folder name (title) to the CSV
                    writer.writerow([nom_complet_sous_dossier])
                    print(f"  Wrote folder title to CSV: {nom_complet_sous_dossier}")

                    # 2. Find and Sort Image Chunks
                    # ... (Chunk finding logic remains the same) ...
                    try:
                        all_files = [f for f in os.listdir(TARGET_FOLDER_PATH) if f.lower().endswith(".png")]
                        sorted_chunks = sorted(all_files, key=natural_sort_key)
                    except OSError as e:
                        print(f"  🛑 Error listing files in {TARGET_FOLDER_PATH}: {e}")
                        sorted_chunks = []


                    if not sorted_chunks:
                        print("  No PNG chunk files found in this folder.")
                    else:
                        print(f"  Found {len(sorted_chunks)} chunk(s) to process.")

                        # --- Variables to store previous OCR results ---
                        ocr_result_n_minus_1 = ""
                        ocr_result_n_minus_2 = ""

                        # --- Loop through Image Chunks ---
                        # ... (The rest of the loop logic remains exactly the same as the previous version) ...
                        for i, chunk_filename in enumerate(tqdm(sorted_chunks, desc="Processing Chunks")):
                            chunk_path = os.path.join(TARGET_FOLDER_PATH, chunk_filename)
                            gemini_file = None
                            current_ocr_result = "" # Initialize for this chunk

                            print(f"\n    Processing chunk {i+1}/{len(sorted_chunks)}: {chunk_filename}")

                            try:
                                # 3. Select System Prompt and Prepare Context
                                prompt_parts = []
                                if i == 0:
                                    # First chunk: No preceding context
                                    current_system_instruction = SYSTEM_INSTRUCTION_FIRST_CHUNK.format(sujet_copie=sujet_copie)
                                    print("      Using FIRST chunk system prompt.")
                                else:
                                    # Subsequent chunks: Prepare context text
                                    current_system_instruction = SYSTEM_INSTRUCTION_WITH_CONTEXT.format(sujet_copie=sujet_copie)
                                    context_text = "\n--- Texte Précédent ---\n"
                                    if ocr_result_n_minus_2: # Add N-2 if available
                                        context_text += ocr_result_n_minus_2 + "\n" # Add newline separator
                                    if ocr_result_n_minus_1: # Add N-1 if available
                                        context_text += ocr_result_n_minus_1
                                    context_text += "\n--- Fin du Texte Précédent ---\n\n--- Image Actuelle à Transcrire ---"
                                    # Add context text as the first part of the prompt
                                    prompt_parts.append(context_text)
                                    print(f"      Using CONTEXT system prompt (Context length: ~{len(context_text)} chars).")


                                # 4. Upload Image Chunk
                                print(f"      Uploading: {chunk_filename}...")
                                gemini_file = upload_to_gemini(chunk_path)
                                if gemini_file is None:
                                    current_ocr_result = f"ERROR: Upload failed for {chunk_filename}"
                                    writer.writerow([current_ocr_result])
                                    print(f"      {current_ocr_result}")
                                    continue # Skip to next chunk

                                # Add the image file as the next part of the prompt
                                prompt_parts.append(gemini_file)

                                # 5. Initialize Model (Can potentially be outside loop if system prompt doesn't change often, but safer inside for now)
                                # Re-create model instance ensures the correct system prompt is used
                                model = genai.GenerativeModel(
                                    model_name=MODEL_NAME,
                                    generation_config=GENERATION_CONFIG,
                                    system_instruction=current_system_instruction,
                                )

                                # 6. Call Gemini API
                                print("      Calling Gemini API...")
                                response = model.generate_content(prompt_parts, request_options={'timeout': 600}) # Increased timeout

                                # 7. Extract OCR Result
                                if response and response.parts:
                                     try:
                                         current_ocr_result = response.text
                                         # Optional: Remove potential leading/trailing whitespace
                                         current_ocr_result = current_ocr_result.strip()
                                         print(f"      OCR result received (~{len(current_ocr_result)} chars).")
                                     except ValueError:
                                         print(f"      ⚠️ Warning: Could not extract text from response for {chunk_filename}. Response: {response.prompt_feedback}")
                                         current_ocr_result = f"ERROR: No text found or content blocked ({response.prompt_feedback})"
                                     except Exception as e_resp:
                                        print(f"      🛑 Error extracting text from response: {e_resp}")
                                        current_ocr_result = f"ERROR: Response parsing failed - {e_resp}"
                                else:
                                    print(f"      ⚠️ Warning: Empty or invalid response received for {chunk_filename}.")
                                    current_ocr_result = "ERROR: Empty or invalid response from API"

                                # Write result to CSV
                                writer.writerow([current_ocr_result])
                                # print(f"      Wrote OCR result to CSV.")

                                # 8. Update History for Next Iteration
                                ocr_result_n_minus_2 = ocr_result_n_minus_1
                                ocr_result_n_minus_1 = current_ocr_result # Store the raw result from this chunk

                            except Exception as e_api:
                                print(f"    🛑 An error occurred during API call or processing for {chunk_filename}:")
                                traceback.print_exc() # Print full traceback for debugging
                                error_message = f"ERROR: API call failed for {chunk_filename} - {type(e_api).__name__}"
                                writer.writerow([error_message])
                                # Update history with error marker to avoid propagating bad context
                                ocr_result_n_minus_2 = ocr_result_n_minus_1
                                ocr_result_n_minus_1 = error_message # Store error marker

                            finally:
                                # 9. Delete Uploaded File from Gemini
                                if gemini_file:
                                    delete_gemini_file(gemini_file)
                                time.sleep(1.5) # Slightly increased delay


            except IOError as e:
                print(f"🛑 Error opening or writing to CSV file {OUTPUT_CSV_FILE}: {e}")
            except Exception as e_main:
                 print(f"🛑 An unexpected error occurred during the main process:")
                 traceback.print_exc()

            print("\n--- OCR Process Complete for Folder ---")
            print(f"Results saved in: {OUTPUT_CSV_FILE}") # Display the final generated path

Gemini API Key configured.
Starting OCR process for single folder: 'Le bonheur est-il affaire de hasard ou de nécessité ? (AGREG interne 2022 - note : 8)'
  Subject detected: 'Le bonheur est-il affaire de hasard ou de nécessité ?'
  Output CSV will be saved as: '/content/Le bonheur est-il affaire de hasard ou de nécessité _.csv'
  CSV file opened successfully.
  Wrote folder title to CSV: Le bonheur est-il affaire de hasard ou de nécessité ? (AGREG interne 2022 - note : 8)
  Found 56 chunk(s) to process.


Processing Chunks:   0%|          | 0/56 [00:00<?, ?it/s]


    Processing chunk 1/56: 0001.png
      Using FIRST chunk system prompt.
      Uploading: 0001.png...
      Calling Gemini API...
      OCR result received (~542 chars).

    Processing chunk 2/56: 0002.png
      Using CONTEXT system prompt (Context length: ~635 chars).
      Uploading: 0002.png...
      Calling Gemini API...
      OCR result received (~647 chars).

    Processing chunk 3/56: 0003.png
      Using CONTEXT system prompt (Context length: ~1283 chars).
      Uploading: 0003.png...
      Calling Gemini API...
      OCR result received (~257 chars).

    Processing chunk 4/56: 0004.png
      Using CONTEXT system prompt (Context length: ~998 chars).
      Uploading: 0004.png...
      Calling Gemini API...
      OCR result received (~637 chars).

    Processing chunk 5/56: 0005.png
      Using CONTEXT system prompt (Context length: ~988 chars).
      Uploading: 0005.png...
      Calling Gemini API...
      OCR result received (~604 chars).

    Processing chunk 6/56: 0006.p

In [ ]:
import shutil
import os

src = '/content/drive/MyDrive/folder_partitioned_merged/Le bonheur est-il affaire de hasard ou de nécessité ? (AGREG interne 2022 - note : 8)'
dst = '/content/Le bonheur affaire de hasard ou de nécessité (attention)'

if os.path.exists(dst):
    shutil.rmtree(dst)

shutil.copytree(src, dst)

'/content/Le bonheur affaire de hasard ou de nécessité (attention)'

In [ ]:
import shutil
import os

src = '/content/drive/MyDrive/folder_partitioned_merged/Peut-on vivre en paix avec son inconscient ? (AGREG interne 2021 - note : 14) (1)'
dst = '/content/Peut-on vivre en paix avec son inconscient ? (AGREG interne 2021 - note : 14) (1)'

if os.path.exists(dst):
    shutil.rmtree(dst)

shutil.copytree(src, dst)

'/content/Peut-on vivre en paix avec son inconscient ? (AGREG interne 2021 - note : 14) (1)'

In [ ]:
#@title OCR avec enrengistrement en CSV au même nom

import os
import google.generativeai as genai
import csv
import time
import re # For natural sorting and filename sanitization
from tqdm.notebook import tqdm # Progress bars
import traceback # For detailed error printing

# --- Configuration ---
# *** SET THE TARGET FOLDER HERE ***
TARGET_FOLDER_PATH = "/content/Peut-on vivre en paix avec son inconscient  - bis? (AGREG interne 2021 - note : 14) (1)" # Example: Set to the specific subfolder path you want to process

# Output CSV file path will be generated dynamically later

# Gemini API Key (ensure it's set as an environment variable)
# ... (API Key configuration remains the same) ...
try:
    genai.configure(api_key=os.environ["GEMINI_API_KEY"])
    print("Gemini API Key configured.")
except KeyError:
    print("🛑 Error: GEMINI_API_KEY environment variable not set.")
    # sys.exit(1) # Optional: exit if running unattended
except Exception as e:
    print(f"🛑 Error configuring Gemini API: {e}")
    # sys.exit(1) # Optional: exit if running unattended


# Gemini Model Configuration)
GENERATION_CONFIG = {
  "temperature": 0.2, # Slightly lower temp might encourage more fidelity
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}
MODEL_NAME = "gemini-2.5-pro-preview-03-25"


# System Prompts (remain the same)
SYSTEM_INSTRUCTION_WITH_CONTEXT = """Tâche Principale : Recopie aussi fidèlement que possible le texte manuscrit de l'image fournie. Répond **exclusivement** avec le texte transcrit, sans aucune introduction, commentaire, ou note.

Contexte de la Dissertation : Ce texte est un extrait d'une dissertation de philosophie portant sur le sujet "{sujet_copie}".

Mise en forme :
- Si des mots ou groupes de mots sont soulignés dans l'image, mets-les en italique en markdown (*texte souligné*).
- Si un mot est barré, encadre-le de tildes en markdown (~mot barré~).

Aide Contextuelle (Texte Précédent) : Pour t'aider à déchiffrer l'écriture, le texte qui précède *immédiatement* celui de l'image actuelle est fourni ci-dessous (séparé par '--- Texte Précédent ---'). Ce texte a été obtenu par OCR des sections précédentes de la dissertation. Le texte de l'image actuelle commence là où le texte précédent s'arrête (il peut y avoir un léger chevauchement de lignes dû au découpage des images).

**Priorité Absolue :** Ta tâche principale reste de transcrire *exactement* ce que tu vois sur l'image actuelle. Utilise le texte précédent **uniquement** comme une aide *secondaire* pour résoudre des ambiguïtés ou déchiffrer des mots très difficiles. **Ne laisse PAS le texte précédent remplacer ou modifier ce qui est clairement visible sur l'image actuelle.** En cas de conflit entre le texte précédent et l'image, **la transcription fidèle de l'image PRÉVAUT.**
"""

SYSTEM_INSTRUCTION_FIRST_CHUNK = """Tâche Principale : Recopie aussi fidèlement que possible le texte manuscrit de l'image fournie. Répond **exclusivement** avec le texte transcrit, sans aucune introduction, commentaire, ou note.

Contexte de la Dissertation : Ce texte est un extrait d'une dissertation de philosophie portant sur le sujet "{sujet_copie}". Ceci est le tout début de la dissertation.

Mise en forme :
- Si des mots ou groupes de mots sont soulignés dans l'image, mets-les en italique en markdown (*texte souligné*).
- Si un mot est barré, encadre-le de tildes en markdown (~mot barré~).
"""

# --- Helper Functions ---

def extract_folder_info(folder_path):
    """Extracts the subject and full name from a folder path name."""
    if not os.path.isdir(folder_path): return None, None
    full_name = os.path.basename(folder_path)
    subject = None
    index_parenthese = full_name.find('(')
    if index_parenthese != -1: subject = full_name[:index_parenthese].strip()
    else: subject = full_name.strip() # Fallback
    return subject, full_name

# --- NEW: Function to sanitize filename ---
def sanitize_filename(filename):
    """Removes or replaces characters invalid for filenames."""
    # Remove leading/trailing whitespace
    sanitized = filename.strip()
    # Replace common invalid characters with underscore (adjust as needed)
    sanitized = re.sub(r'[\\/*?:"<>|]', '_', sanitized)
    # Replace forward slash specifically (common in paths)
    sanitized = sanitized.replace('/', '_')
    # Optional: Limit length if necessary (e.g., 200 chars)
    # max_len = 200
    # if len(sanitized) > max_len:
    #     name, ext = os.path.splitext(sanitized)
    #     sanitized = name[:max_len - len(ext) - 1] + '~' + ext
    return sanitized


# ... (upload_to_gemini, delete_gemini_file, natural_sort_key remain the same) ...
def upload_to_gemini(path, mime_type="image/png"):
    """Uploads a file to Gemini and returns the File object."""
    try:
        file = genai.upload_file(path, mime_type=mime_type)
        return file
    except Exception as e:
        print(f"  🛑 Error uploading file {path}: {e}")
        return None

def delete_gemini_file(file_object):
    """Deletes a file from Gemini service."""
    if file_object:
        try: genai.delete_file(file_object.name)
        except Exception as e: print(f"  ⚠️ Warning: Failed to delete file {file_object.name}: {e}")

def natural_sort_key(s):
    """Key for sorting strings containing numbers naturally."""
    return [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', s)]


# --- Main Processing Logic for Single Folder ---

if __name__ == "__main__":
    if not os.path.isdir(TARGET_FOLDER_PATH):
        print(f"🛑 Error: Target folder not found at '{TARGET_FOLDER_PATH}'")
    else:
        print(f"Starting OCR process for single folder: '{os.path.basename(TARGET_FOLDER_PATH)}'")

        # 1. Extract Subject and Full Name for the target folder
        sujet_copie, nom_complet_sous_dossier = extract_folder_info(TARGET_FOLDER_PATH)

        if sujet_copie is None:
            print(f"🛑 Error: Could not extract info from folder path '{TARGET_FOLDER_PATH}'. Cannot proceed.")
        else:
            print(f"  Subject detected: '{sujet_copie}'")

            # --- GENERATE DYNAMIC OUTPUT CSV FILENAME ---
            parent_dir = os.path.dirname(TARGET_FOLDER_PATH) # Get directory containing the target folder
            sanitized_subject = sanitize_filename(sujet_copie) # Sanitize the subject for use in filename
            OUTPUT_CSV_FILE = os.path.join(parent_dir, f"{sanitized_subject}.csv")
            print(f"  Output CSV will be saved as: '{OUTPUT_CSV_FILE}'")
            # --- End Filename Generation ---

            # --- Initialize CSV Writer ---
            try:
                with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as csvfile:
                    writer = csv.writer(csvfile)
                    print("  CSV file opened successfully.")

                    # Write the folder name (title) to the CSV
                    writer.writerow([nom_complet_sous_dossier])
                    print(f"  Wrote folder title to CSV: {nom_complet_sous_dossier}")

                    # 2. Find and Sort Image Chunks
                    # ... (Chunk finding logic remains the same) ...
                    try:
                        all_files = [f for f in os.listdir(TARGET_FOLDER_PATH) if f.lower().endswith(".png")]
                        sorted_chunks = sorted(all_files, key=natural_sort_key)
                    except OSError as e:
                        print(f"  🛑 Error listing files in {TARGET_FOLDER_PATH}: {e}")
                        sorted_chunks = []


                    if not sorted_chunks:
                        print("  No PNG chunk files found in this folder.")
                    else:
                        print(f"  Found {len(sorted_chunks)} chunk(s) to process.")

                        # --- Variables to store previous OCR results ---
                        ocr_result_n_minus_1 = ""
                        ocr_result_n_minus_2 = ""

                        # --- Loop through Image Chunks ---
                        # ... (The rest of the loop logic remains exactly the same as the previous version) ...
                        for i, chunk_filename in enumerate(tqdm(sorted_chunks, desc="Processing Chunks")):
                            chunk_path = os.path.join(TARGET_FOLDER_PATH, chunk_filename)
                            gemini_file = None
                            current_ocr_result = "" # Initialize for this chunk

                            print(f"\n    Processing chunk {i+1}/{len(sorted_chunks)}: {chunk_filename}")

                            try:
                                # 3. Select System Prompt and Prepare Context
                                prompt_parts = []
                                if i == 0:
                                    # First chunk: No preceding context
                                    current_system_instruction = SYSTEM_INSTRUCTION_FIRST_CHUNK.format(sujet_copie=sujet_copie)
                                    print("      Using FIRST chunk system prompt.")
                                else:
                                    # Subsequent chunks: Prepare context text
                                    current_system_instruction = SYSTEM_INSTRUCTION_WITH_CONTEXT.format(sujet_copie=sujet_copie)
                                    context_text = "\n--- Texte Précédent ---\n"
                                    if ocr_result_n_minus_2: # Add N-2 if available
                                        context_text += ocr_result_n_minus_2 + "\n" # Add newline separator
                                    if ocr_result_n_minus_1: # Add N-1 if available
                                        context_text += ocr_result_n_minus_1
                                    context_text += "\n--- Fin du Texte Précédent ---\n\n--- Image Actuelle à Transcrire ---"
                                    # Add context text as the first part of the prompt
                                    prompt_parts.append(context_text)
                                    print(f"      Using CONTEXT system prompt (Context length: ~{len(context_text)} chars).")


                                # 4. Upload Image Chunk
                                print(f"      Uploading: {chunk_filename}...")
                                gemini_file = upload_to_gemini(chunk_path)
                                if gemini_file is None:
                                    current_ocr_result = f"ERROR: Upload failed for {chunk_filename}"
                                    writer.writerow([current_ocr_result])
                                    print(f"      {current_ocr_result}")
                                    continue # Skip to next chunk

                                # Add the image file as the next part of the prompt
                                prompt_parts.append(gemini_file)

                                # 5. Initialize Model (Can potentially be outside loop if system prompt doesn't change often, but safer inside for now)
                                # Re-create model instance ensures the correct system prompt is used
                                model = genai.GenerativeModel(
                                    model_name=MODEL_NAME,
                                    generation_config=GENERATION_CONFIG,
                                    system_instruction=current_system_instruction,
                                )

                                # 6. Call Gemini API
                                print("      Calling Gemini API...")
                                response = model.generate_content(prompt_parts, request_options={'timeout': 600}) # Increased timeout

                                # 7. Extract OCR Result
                                if response and response.parts:
                                     try:
                                         current_ocr_result = response.text
                                         # Optional: Remove potential leading/trailing whitespace
                                         current_ocr_result = current_ocr_result.strip()
                                         print(f"      OCR result received (~{len(current_ocr_result)} chars).")
                                     except ValueError:
                                         print(f"      ⚠️ Warning: Could not extract text from response for {chunk_filename}. Response: {response.prompt_feedback}")
                                         current_ocr_result = f"ERROR: No text found or content blocked ({response.prompt_feedback})"
                                     except Exception as e_resp:
                                        print(f"      🛑 Error extracting text from response: {e_resp}")
                                        current_ocr_result = f"ERROR: Response parsing failed - {e_resp}"
                                else:
                                    print(f"      ⚠️ Warning: Empty or invalid response received for {chunk_filename}.")
                                    current_ocr_result = "ERROR: Empty or invalid response from API"

                                # Write result to CSV
                                writer.writerow([current_ocr_result])
                                # print(f"      Wrote OCR result to CSV.")

                                # 8. Update History for Next Iteration
                                ocr_result_n_minus_2 = ocr_result_n_minus_1
                                ocr_result_n_minus_1 = current_ocr_result # Store the raw result from this chunk

                            except Exception as e_api:
                                print(f"    🛑 An error occurred during API call or processing for {chunk_filename}:")
                                traceback.print_exc() # Print full traceback for debugging
                                error_message = f"ERROR: API call failed for {chunk_filename} - {type(e_api).__name__}"
                                writer.writerow([error_message])
                                # Update history with error marker to avoid propagating bad context
                                ocr_result_n_minus_2 = ocr_result_n_minus_1
                                ocr_result_n_minus_1 = error_message # Store error marker

                            finally:
                                # 9. Delete Uploaded File from Gemini
                                if gemini_file:
                                    delete_gemini_file(gemini_file)
                                time.sleep(1.5) # Slightly increased delay


            except IOError as e:
                print(f"🛑 Error opening or writing to CSV file {OUTPUT_CSV_FILE}: {e}")
            except Exception as e_main:
                 print(f"🛑 An unexpected error occurred during the main process:")
                 traceback.print_exc()

            print("\n--- OCR Process Complete for Folder ---")
            print(f"Results saved in: {OUTPUT_CSV_FILE}") # Display the final generated path

Gemini API Key configured.
Starting OCR process for single folder: 'Peut-on vivre en paix avec son inconscient  - bis? (AGREG interne 2021 - note : 14) (1)'
  Subject detected: 'Peut-on vivre en paix avec son inconscient  - bis?'
  Output CSV will be saved as: '/content/Peut-on vivre en paix avec son inconscient  - bis_.csv'
  CSV file opened successfully.
  Wrote folder title to CSV: Peut-on vivre en paix avec son inconscient  - bis? (AGREG interne 2021 - note : 14) (1)
  Found 43 chunk(s) to process.


Processing Chunks:   0%|          | 0/43 [00:00<?, ?it/s]


    Processing chunk 1/43: 0001.png
      Using FIRST chunk system prompt.
      Uploading: 0001.png...
      Calling Gemini API...
      OCR result received (~836 chars).
  ⚠️ Warning: Failed to delete file files/xw7pgnmaux9z: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

    Processing chunk 2/43: 0002.png
      Using CONTEXT system prompt (Context length: ~929 chars).
      Uploading: 0002.png...
      Calling Gemini API...
      OCR result received (~823 chars).

    Processing chunk 3/43: 0003.png
      Using CONTEXT system prompt (Context length: ~1753 chars).
      Uploading: 0003.png...
      Calling Gemini API...
      OCR result received (~403 chars).

    Processing chunk 4/43: 0004.png
      Using CONTEXT system prompt (Context length: ~1320 chars).
      Uploading: 0004.png...
      Calling Gemini API...
      OCR result received (~717 chars).
  ⚠️ Warning: Failed to delete file files/aodefgar70kc: ('Connection aborted.', Re

In [ ]:
import shutil
import os

src = '/content/drive/MyDrive/folder_partitioned_merged/Le bonheur est-il affaire de hasard et de nécessité ? (AGREG interne -  note : 15,5)'
dst = '/content/Le bonheur est-il affaire de hasard et de nécessité ? (AGREG interne -  note : 15,5)'

if os.path.exists(dst):
    shutil.rmtree(dst)

shutil.copytree(src, dst)

'/content/Le bonheur est-il affaire de hasard et de nécessité ? (AGREG interne -  note : 15,5)'

In [ ]:
#@title OCR avec enrengistrement en CSV au même nom

import os
import google.generativeai as genai
import csv
import time
import re # For natural sorting and filename sanitization
from tqdm.notebook import tqdm # Progress bars
import traceback # For detailed error printing

# --- Configuration ---
# *** SET THE TARGET FOLDER HERE ***
TARGET_FOLDER_PATH = "/content/Langage et réalité (AGREG ext. 2018 - note : 19)" # Example: Set to the specific subfolder path you want to process

# Output CSV file path will be generated dynamically later

# Gemini API Key (ensure it's set as an environment variable)
# ... (API Key configuration remains the same) ...
try:
    genai.configure(api_key=os.environ["GEMINI_API_KEY"])
    print("Gemini API Key configured.")
except KeyError:
    print("🛑 Error: GEMINI_API_KEY environment variable not set.")
    # sys.exit(1) # Optional: exit if running unattended
except Exception as e:
    print(f"🛑 Error configuring Gemini API: {e}")
    # sys.exit(1) # Optional: exit if running unattended


# Gemini Model Configuration)
GENERATION_CONFIG = {
  "temperature": 0.2, # Slightly lower temp might encourage more fidelity
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}
MODEL_NAME = "gemini-2.5-pro-preview-03-25"


# System Prompts (remain the same)
SYSTEM_INSTRUCTION_WITH_CONTEXT = """Tâche Principale : Recopie aussi fidèlement que possible le texte manuscrit de l'image fournie. Répond **exclusivement** avec le texte transcrit, sans aucune introduction, commentaire, ou note.

Contexte de la Dissertation : Ce texte est un extrait d'une dissertation de philosophie portant sur le sujet "{sujet_copie}".

Mise en forme :
- Si des mots ou groupes de mots sont soulignés dans l'image, mets-les en italique en markdown (*texte souligné*).
- Si un mot est barré, encadre-le de tildes en markdown (~mot barré~).

Aide Contextuelle (Texte Précédent) : Pour t'aider à déchiffrer l'écriture, le texte qui précède *immédiatement* celui de l'image actuelle est fourni ci-dessous (séparé par '--- Texte Précédent ---'). Ce texte a été obtenu par OCR des sections précédentes de la dissertation. Le texte de l'image actuelle commence là où le texte précédent s'arrête (il peut y avoir un léger chevauchement de lignes dû au découpage des images).

**Priorité Absolue :** Ta tâche principale reste de transcrire *exactement* ce que tu vois sur l'image actuelle. Utilise le texte précédent **uniquement** comme une aide *secondaire* pour résoudre des ambiguïtés ou déchiffrer des mots très difficiles. **Ne laisse PAS le texte précédent remplacer ou modifier ce qui est clairement visible sur l'image actuelle.** En cas de conflit entre le texte précédent et l'image, **la transcription fidèle de l'image PRÉVAUT.**
"""

SYSTEM_INSTRUCTION_FIRST_CHUNK = """Tâche Principale : Recopie aussi fidèlement que possible le texte manuscrit de l'image fournie. Répond **exclusivement** avec le texte transcrit, sans aucune introduction, commentaire, ou note.

Contexte de la Dissertation : Ce texte est un extrait d'une dissertation de philosophie portant sur le sujet "{sujet_copie}". Ceci est le tout début de la dissertation.

Mise en forme :
- Si des mots ou groupes de mots sont soulignés dans l'image, mets-les en italique en markdown (*texte souligné*).
- Si un mot est barré, encadre-le de tildes en markdown (~mot barré~).
"""

# --- Helper Functions ---

def extract_folder_info(folder_path):
    """Extracts the subject and full name from a folder path name."""
    if not os.path.isdir(folder_path): return None, None
    full_name = os.path.basename(folder_path)
    subject = None
    index_parenthese = full_name.find('(')
    if index_parenthese != -1: subject = full_name[:index_parenthese].strip()
    else: subject = full_name.strip() # Fallback
    return subject, full_name

# --- NEW: Function to sanitize filename ---
def sanitize_filename(filename):
    """Removes or replaces characters invalid for filenames."""
    # Remove leading/trailing whitespace
    sanitized = filename.strip()
    # Replace common invalid characters with underscore (adjust as needed)
    sanitized = re.sub(r'[\\/*?:"<>|]', '_', sanitized)
    # Replace forward slash specifically (common in paths)
    sanitized = sanitized.replace('/', '_')
    # Optional: Limit length if necessary (e.g., 200 chars)
    # max_len = 200
    # if len(sanitized) > max_len:
    #     name, ext = os.path.splitext(sanitized)
    #     sanitized = name[:max_len - len(ext) - 1] + '~' + ext
    return sanitized


# ... (upload_to_gemini, delete_gemini_file, natural_sort_key remain the same) ...
def upload_to_gemini(path, mime_type="image/png"):
    """Uploads a file to Gemini and returns the File object."""
    try:
        file = genai.upload_file(path, mime_type=mime_type)
        return file
    except Exception as e:
        print(f"  🛑 Error uploading file {path}: {e}")
        return None

def delete_gemini_file(file_object):
    """Deletes a file from Gemini service."""
    if file_object:
        try: genai.delete_file(file_object.name)
        except Exception as e: print(f"  ⚠️ Warning: Failed to delete file {file_object.name}: {e}")

def natural_sort_key(s):
    """Key for sorting strings containing numbers naturally."""
    return [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', s)]


# --- Main Processing Logic for Single Folder ---

if __name__ == "__main__":
    if not os.path.isdir(TARGET_FOLDER_PATH):
        print(f"🛑 Error: Target folder not found at '{TARGET_FOLDER_PATH}'")
    else:
        print(f"Starting OCR process for single folder: '{os.path.basename(TARGET_FOLDER_PATH)}'")

        # 1. Extract Subject and Full Name for the target folder
        sujet_copie, nom_complet_sous_dossier = extract_folder_info(TARGET_FOLDER_PATH)

        if sujet_copie is None:
            print(f"🛑 Error: Could not extract info from folder path '{TARGET_FOLDER_PATH}'. Cannot proceed.")
        else:
            print(f"  Subject detected: '{sujet_copie}'")

            # --- GENERATE DYNAMIC OUTPUT CSV FILENAME ---
            parent_dir = os.path.dirname(TARGET_FOLDER_PATH) # Get directory containing the target folder
            sanitized_subject = sanitize_filename(sujet_copie) # Sanitize the subject for use in filename
            OUTPUT_CSV_FILE = os.path.join(parent_dir, f"{sanitized_subject}.csv")
            print(f"  Output CSV will be saved as: '{OUTPUT_CSV_FILE}'")
            # --- End Filename Generation ---

            # --- Initialize CSV Writer ---
            try:
                with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as csvfile:
                    writer = csv.writer(csvfile)
                    print("  CSV file opened successfully.")

                    # Write the folder name (title) to the CSV
                    writer.writerow([nom_complet_sous_dossier])
                    print(f"  Wrote folder title to CSV: {nom_complet_sous_dossier}")

                    # 2. Find and Sort Image Chunks
                    # ... (Chunk finding logic remains the same) ...
                    try:
                        all_files = [f for f in os.listdir(TARGET_FOLDER_PATH) if f.lower().endswith(".png")]
                        sorted_chunks = sorted(all_files, key=natural_sort_key)
                    except OSError as e:
                        print(f"  🛑 Error listing files in {TARGET_FOLDER_PATH}: {e}")
                        sorted_chunks = []


                    if not sorted_chunks:
                        print("  No PNG chunk files found in this folder.")
                    else:
                        print(f"  Found {len(sorted_chunks)} chunk(s) to process.")

                        # --- Variables to store previous OCR results ---
                        ocr_result_n_minus_1 = ""
                        ocr_result_n_minus_2 = ""

                        # --- Loop through Image Chunks ---
                        # ... (The rest of the loop logic remains exactly the same as the previous version) ...
                        for i, chunk_filename in enumerate(tqdm(sorted_chunks, desc="Processing Chunks")):
                            chunk_path = os.path.join(TARGET_FOLDER_PATH, chunk_filename)
                            gemini_file = None
                            current_ocr_result = "" # Initialize for this chunk

                            print(f"\n    Processing chunk {i+1}/{len(sorted_chunks)}: {chunk_filename}")

                            try:
                                # 3. Select System Prompt and Prepare Context
                                prompt_parts = []
                                if i == 0:
                                    # First chunk: No preceding context
                                    current_system_instruction = SYSTEM_INSTRUCTION_FIRST_CHUNK.format(sujet_copie=sujet_copie)
                                    print("      Using FIRST chunk system prompt.")
                                else:
                                    # Subsequent chunks: Prepare context text
                                    current_system_instruction = SYSTEM_INSTRUCTION_WITH_CONTEXT.format(sujet_copie=sujet_copie)
                                    context_text = "\n--- Texte Précédent ---\n"
                                    if ocr_result_n_minus_2: # Add N-2 if available
                                        context_text += ocr_result_n_minus_2 + "\n" # Add newline separator
                                    if ocr_result_n_minus_1: # Add N-1 if available
                                        context_text += ocr_result_n_minus_1
                                    context_text += "\n--- Fin du Texte Précédent ---\n\n--- Image Actuelle à Transcrire ---"
                                    # Add context text as the first part of the prompt
                                    prompt_parts.append(context_text)
                                    print(f"      Using CONTEXT system prompt (Context length: ~{len(context_text)} chars).")


                                # 4. Upload Image Chunk
                                print(f"      Uploading: {chunk_filename}...")
                                gemini_file = upload_to_gemini(chunk_path)
                                if gemini_file is None:
                                    current_ocr_result = f"ERROR: Upload failed for {chunk_filename}"
                                    writer.writerow([current_ocr_result])
                                    print(f"      {current_ocr_result}")
                                    continue # Skip to next chunk

                                # Add the image file as the next part of the prompt
                                prompt_parts.append(gemini_file)

                                # 5. Initialize Model (Can potentially be outside loop if system prompt doesn't change often, but safer inside for now)
                                # Re-create model instance ensures the correct system prompt is used
                                model = genai.GenerativeModel(
                                    model_name=MODEL_NAME,
                                    generation_config=GENERATION_CONFIG,
                                    system_instruction=current_system_instruction,
                                )

                                # 6. Call Gemini API
                                print("      Calling Gemini API...")
                                response = model.generate_content(prompt_parts, request_options={'timeout': 600}) # Increased timeout

                                # 7. Extract OCR Result
                                if response and response.parts:
                                     try:
                                         current_ocr_result = response.text
                                         # Optional: Remove potential leading/trailing whitespace
                                         current_ocr_result = current_ocr_result.strip()
                                         print(f"      OCR result received (~{len(current_ocr_result)} chars).")
                                     except ValueError:
                                         print(f"      ⚠️ Warning: Could not extract text from response for {chunk_filename}. Response: {response.prompt_feedback}")
                                         current_ocr_result = f"ERROR: No text found or content blocked ({response.prompt_feedback})"
                                     except Exception as e_resp:
                                        print(f"      🛑 Error extracting text from response: {e_resp}")
                                        current_ocr_result = f"ERROR: Response parsing failed - {e_resp}"
                                else:
                                    print(f"      ⚠️ Warning: Empty or invalid response received for {chunk_filename}.")
                                    current_ocr_result = "ERROR: Empty or invalid response from API"

                                # Write result to CSV
                                writer.writerow([current_ocr_result])
                                # print(f"      Wrote OCR result to CSV.")

                                # 8. Update History for Next Iteration
                                ocr_result_n_minus_2 = ocr_result_n_minus_1
                                ocr_result_n_minus_1 = current_ocr_result # Store the raw result from this chunk

                            except Exception as e_api:
                                print(f"    🛑 An error occurred during API call or processing for {chunk_filename}:")
                                traceback.print_exc() # Print full traceback for debugging
                                error_message = f"ERROR: API call failed for {chunk_filename} - {type(e_api).__name__}"
                                writer.writerow([error_message])
                                # Update history with error marker to avoid propagating bad context
                                ocr_result_n_minus_2 = ocr_result_n_minus_1
                                ocr_result_n_minus_1 = error_message # Store error marker

                            finally:
                                # 9. Delete Uploaded File from Gemini
                                if gemini_file:
                                    delete_gemini_file(gemini_file)
                                time.sleep(1.5) # Slightly increased delay


            except IOError as e:
                print(f"🛑 Error opening or writing to CSV file {OUTPUT_CSV_FILE}: {e}")
            except Exception as e_main:
                 print(f"🛑 An unexpected error occurred during the main process:")
                 traceback.print_exc()

            print("\n--- OCR Process Complete for Folder ---")
            print(f"Results saved in: {OUTPUT_CSV_FILE}") # Display the final generated path

Gemini API Key configured.
Starting OCR process for single folder: 'Langage et réalité (AGREG ext. 2018 - note : 19)'
  Subject detected: 'Langage et réalité'
  Output CSV will be saved as: '/content/Langage et réalité.csv'
  CSV file opened successfully.
  Wrote folder title to CSV: Langage et réalité (AGREG ext. 2018 - note : 19)
  Found 52 chunk(s) to process.


Processing Chunks:   0%|          | 0/52 [00:00<?, ?it/s]


    Processing chunk 1/52: 0001.png
      Using FIRST chunk system prompt.
      Uploading: 0001.png...
      Calling Gemini API...
      OCR result received (~945 chars).

    Processing chunk 2/52: 0002.png
      Using CONTEXT system prompt (Context length: ~1038 chars).
      Uploading: 0002.png...
      Calling Gemini API...
      OCR result received (~861 chars).

    Processing chunk 3/52: 0003.png
      Using CONTEXT system prompt (Context length: ~1900 chars).
      Uploading: 0003.png...
      Calling Gemini API...
      OCR result received (~438 chars).

    Processing chunk 4/52: 0004.png
      Using CONTEXT system prompt (Context length: ~1393 chars).
      Uploading: 0004.png...
      Calling Gemini API...
      OCR result received (~873 chars).

    Processing chunk 5/52: 0005.png
      Using CONTEXT system prompt (Context length: ~1405 chars).
      Uploading: 0005.png...
      Calling Gemini API...
      OCR result received (~835 chars).

    Processing chunk 6/52: 000

Traceback (most recent call last):
  File "<ipython-input-51-f026a913f01f>", line 218, in <cell line: 0>
    response = model.generate_content(prompt_parts, request_options={'timeout': 600}) # Increased timeout
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/generativeai/generative_models.py", line 331, in generate_content
    response = self._client.generate_content(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/ai/generativelanguage_v1beta/services/generative_service/client.py", line 835, in generate_content
    response = rpc(
               ^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/api_core/gapic_v1/method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/api_core/retry/retry_unary.py", line 293, in retry_w

    🛑 An error occurred during API call or processing for 0032.png:

    Processing chunk 33/52: 0033.png
      Using CONTEXT system prompt (Context length: ~533 chars).
      Uploading: 0033.png...
      Calling Gemini API...


...

Traceback (most recent call last):
  File "<ipython-input-51-f026a913f01f>", line 218, in <cell line: 0>
    response = model.generate_content(prompt_parts, request_options={'timeout': 600}) # Increased timeout
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/generativeai/generative_models.py", line 331, in generate_content
    response = self._client.generate_content(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/ai/generativelanguage_v1beta/services/generative_service/client.py", line 835, in generate_content
    response = rpc(
               ^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/api_core/gapic_v1/method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/api_core/retry/retry_unary.py", line 293, in retry_w

    🛑 An error occurred during API call or processing for 0041.png:

    Processing chunk 42/52: 0042.png
      Using CONTEXT system prompt (Context length: ~200 chars).
      Uploading: 0042.png...
      Calling Gemini API...


Traceback (most recent call last):
  File "<ipython-input-51-f026a913f01f>", line 218, in <cell line: 0>
    response = model.generate_content(prompt_parts, request_options={'timeout': 600}) # Increased timeout
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/generativeai/generative_models.py", line 331, in generate_content
    response = self._client.generate_content(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/ai/generativelanguage_v1beta/services/generative_service/client.py", line 835, in generate_content
    response = rpc(
               ^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/api_core/gapic_v1/method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/api_core/retry/retry_unary.py", line 293, in retry_w

    🛑 An error occurred during API call or processing for 0042.png:


KeyboardInterrupt: 

In [ ]:
import shutil
import os

src = '/content/drive/MyDrive/folder_partitioned_merged/Langage et réalité (AGREG ext. 2018 - note : 19)'
dst = '/content/Langage et réalité (AGREG ext. 2018 - note : 19)'

if os.path.exists(dst):
    shutil.rmtree(dst)

shutil.copytree(src, dst)

'/content/Langage et réalité (AGREG ext. 2018 - note : 19)'

In [ ]:
#@title OCR avec enrengistrement en CSV au même nom

import os
import google.generativeai as genai
import csv
import time
import re # For natural sorting and filename sanitization
from tqdm.notebook import tqdm # Progress bars
import traceback # For detailed error printing

# --- Configuration ---
# *** SET THE TARGET FOLDER HERE ***
TARGET_FOLDER_PATH = "/content/Langage et réalité (AGREG ext. 2018 - note : 19)" # Example: Set to the specific subfolder path you want to process

# Output CSV file path will be generated dynamically later

# Gemini API Key (ensure it's set as an environment variable)
# ... (API Key configuration remains the same) ...
try:
    genai.configure(api_key=os.environ["GEMINI_API_KEY"])
    print("Gemini API Key configured.")
except KeyError:
    print("🛑 Error: GEMINI_API_KEY environment variable not set.")
    # sys.exit(1) # Optional: exit if running unattended
except Exception as e:
    print(f"🛑 Error configuring Gemini API: {e}")
    # sys.exit(1) # Optional: exit if running unattended


# Gemini Model Configuration)
GENERATION_CONFIG = {
  "temperature": 0.2, # Slightly lower temp might encourage more fidelity
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}
MODEL_NAME = "gemini-2.5-pro-preview-03-25"


# System Prompts (remain the same)
SYSTEM_INSTRUCTION_WITH_CONTEXT = """Tâche Principale : Recopie aussi fidèlement que possible le texte manuscrit de l'image fournie. Répond **exclusivement** avec le texte transcrit, sans aucune introduction, commentaire, ou note.

Contexte de la Dissertation : Ce texte est un extrait d'une dissertation de philosophie portant sur le sujet "{sujet_copie}".

Mise en forme :
- Si des mots ou groupes de mots sont soulignés dans l'image, mets-les en italique en markdown (*texte souligné*).
- Si un mot est barré, encadre-le de tildes en markdown (~mot barré~).

Aide Contextuelle (Texte Précédent) : Pour t'aider à déchiffrer l'écriture, le texte qui précède *immédiatement* celui de l'image actuelle est fourni ci-dessous (séparé par '--- Texte Précédent ---'). Ce texte a été obtenu par OCR des sections précédentes de la dissertation. Le texte de l'image actuelle commence là où le texte précédent s'arrête (il peut y avoir un léger chevauchement de lignes dû au découpage des images).

**Priorité Absolue :** Ta tâche principale reste de transcrire *exactement* ce que tu vois sur l'image actuelle.
"""

SYSTEM_INSTRUCTION_FIRST_CHUNK = """Tâche Principale : Recopie aussi fidèlement que possible le texte manuscrit de l'image fournie. Répond **exclusivement** avec le texte transcrit, sans aucune introduction, commentaire, ou note.

Contexte de la Dissertation : Ce texte est un extrait d'une dissertation de philosophie portant sur le sujet "{sujet_copie}". Ceci est le tout début de la dissertation.

Mise en forme :
- Si des mots ou groupes de mots sont soulignés dans l'image, mets-les en italique en markdown (*texte souligné*).
- Si un mot est barré, encadre-le de tildes en markdown (~mot barré~).
"""

# --- Helper Functions ---

def extract_folder_info(folder_path):
    """Extracts the subject and full name from a folder path name."""
    if not os.path.isdir(folder_path): return None, None
    full_name = os.path.basename(folder_path)
    subject = None
    index_parenthese = full_name.find('(')
    if index_parenthese != -1: subject = full_name[:index_parenthese].strip()
    else: subject = full_name.strip() # Fallback
    return subject, full_name

# --- NEW: Function to sanitize filename ---
def sanitize_filename(filename):
    """Removes or replaces characters invalid for filenames."""
    # Remove leading/trailing whitespace
    sanitized = filename.strip()
    # Replace common invalid characters with underscore (adjust as needed)
    sanitized = re.sub(r'[\\/*?:"<>|]', '_', sanitized)
    # Replace forward slash specifically (common in paths)
    sanitized = sanitized.replace('/', '_')
    # Optional: Limit length if necessary (e.g., 200 chars)
    # max_len = 200
    # if len(sanitized) > max_len:
    #     name, ext = os.path.splitext(sanitized)
    #     sanitized = name[:max_len - len(ext) - 1] + '~' + ext
    return sanitized


# ... (upload_to_gemini, delete_gemini_file, natural_sort_key remain the same) ...
def upload_to_gemini(path, mime_type="image/png"):
    """Uploads a file to Gemini and returns the File object."""
    try:
        file = genai.upload_file(path, mime_type=mime_type)
        return file
    except Exception as e:
        print(f"  🛑 Error uploading file {path}: {e}")
        return None

def delete_gemini_file(file_object):
    """Deletes a file from Gemini service."""
    if file_object:
        try: genai.delete_file(file_object.name)
        except Exception as e: print(f"  ⚠️ Warning: Failed to delete file {file_object.name}: {e}")

def natural_sort_key(s):
    """Key for sorting strings containing numbers naturally."""
    return [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', s)]


# --- Main Processing Logic for Single Folder ---

if __name__ == "__main__":
    if not os.path.isdir(TARGET_FOLDER_PATH):
        print(f"🛑 Error: Target folder not found at '{TARGET_FOLDER_PATH}'")
    else:
        print(f"Starting OCR process for single folder: '{os.path.basename(TARGET_FOLDER_PATH)}'")

        # 1. Extract Subject and Full Name for the target folder
        sujet_copie, nom_complet_sous_dossier = extract_folder_info(TARGET_FOLDER_PATH)

        if sujet_copie is None:
            print(f"🛑 Error: Could not extract info from folder path '{TARGET_FOLDER_PATH}'. Cannot proceed.")
        else:
            print(f"  Subject detected: '{sujet_copie}'")

            # --- GENERATE DYNAMIC OUTPUT CSV FILENAME ---
            parent_dir = os.path.dirname(TARGET_FOLDER_PATH) # Get directory containing the target folder
            sanitized_subject = sanitize_filename(sujet_copie) # Sanitize the subject for use in filename
            OUTPUT_CSV_FILE = os.path.join(parent_dir, f"{sanitized_subject}.csv")
            print(f"  Output CSV will be saved as: '{OUTPUT_CSV_FILE}'")
            # --- End Filename Generation ---

            # --- Initialize CSV Writer ---
            try:
                with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as csvfile:
                    writer = csv.writer(csvfile)
                    print("  CSV file opened successfully.")

                    # Write the folder name (title) to the CSV
                    writer.writerow([nom_complet_sous_dossier])
                    print(f"  Wrote folder title to CSV: {nom_complet_sous_dossier}")

                    # 2. Find and Sort Image Chunks
                    # ... (Chunk finding logic remains the same) ...
                    try:
                        all_files = [f for f in os.listdir(TARGET_FOLDER_PATH) if f.lower().endswith(".png")]
                        sorted_chunks = sorted(all_files, key=natural_sort_key)
                    except OSError as e:
                        print(f"  🛑 Error listing files in {TARGET_FOLDER_PATH}: {e}")
                        sorted_chunks = []


                    if not sorted_chunks:
                        print("  No PNG chunk files found in this folder.")
                    else:
                        print(f"  Found {len(sorted_chunks)} chunk(s) to process.")

                        # --- Variables to store previous OCR results ---
                        ocr_result_n_minus_1 = ""
                        ocr_result_n_minus_2 = ""

                        # --- Loop through Image Chunks ---
                        # ... (The rest of the loop logic remains exactly the same as the previous version) ...
                        for i, chunk_filename in enumerate(tqdm(sorted_chunks, desc="Processing Chunks")):
                            chunk_path = os.path.join(TARGET_FOLDER_PATH, chunk_filename)
                            gemini_file = None
                            current_ocr_result = "" # Initialize for this chunk

                            print(f"\n    Processing chunk {i+1}/{len(sorted_chunks)}: {chunk_filename}")

                            try:
                                # 3. Select System Prompt and Prepare Context
                                prompt_parts = []
                                if i == 0:
                                    # First chunk: No preceding context
                                    current_system_instruction = SYSTEM_INSTRUCTION_FIRST_CHUNK.format(sujet_copie=sujet_copie)
                                    print("      Using FIRST chunk system prompt.")
                                else:
                                    # Subsequent chunks: Prepare context text
                                    current_system_instruction = SYSTEM_INSTRUCTION_WITH_CONTEXT.format(sujet_copie=sujet_copie)
                                    context_text = "\n--- Texte Précédent ---\n"
                                    if ocr_result_n_minus_2: # Add N-2 if available
                                        context_text += ocr_result_n_minus_2 + "\n" # Add newline separator
                                    if ocr_result_n_minus_1: # Add N-1 if available
                                        context_text += ocr_result_n_minus_1
                                    context_text += "\n--- Fin du Texte Précédent ---\n\n--- Image Actuelle à Transcrire ---"
                                    # Add context text as the first part of the prompt
                                    prompt_parts.append(context_text)
                                    print(f"      Using CONTEXT system prompt (Context length: ~{len(context_text)} chars).")


                                # 4. Upload Image Chunk
                                print(f"      Uploading: {chunk_filename}...")
                                gemini_file = upload_to_gemini(chunk_path)
                                if gemini_file is None:
                                    current_ocr_result = f"ERROR: Upload failed for {chunk_filename}"
                                    writer.writerow([current_ocr_result])
                                    print(f"      {current_ocr_result}")
                                    continue # Skip to next chunk

                                # Add the image file as the next part of the prompt
                                prompt_parts.append(gemini_file)

                                # 5. Initialize Model (Can potentially be outside loop if system prompt doesn't change often, but safer inside for now)
                                # Re-create model instance ensures the correct system prompt is used
                                model = genai.GenerativeModel(
                                    model_name=MODEL_NAME,
                                    generation_config=GENERATION_CONFIG,
                                    system_instruction=current_system_instruction,
                                )

                                # 6. Call Gemini API
                                print("      Calling Gemini API...")
                                response = model.generate_content(prompt_parts, request_options={'timeout': 600}) # Increased timeout

                                # 7. Extract OCR Result
                                if response and response.parts:
                                     try:
                                         current_ocr_result = response.text
                                         # Optional: Remove potential leading/trailing whitespace
                                         current_ocr_result = current_ocr_result.strip()
                                         print(f"      OCR result received (~{len(current_ocr_result)} chars).")
                                     except ValueError:
                                         print(f"      ⚠️ Warning: Could not extract text from response for {chunk_filename}. Response: {response.prompt_feedback}")
                                         current_ocr_result = f"ERROR: No text found or content blocked ({response.prompt_feedback})"
                                     except Exception as e_resp:
                                        print(f"      🛑 Error extracting text from response: {e_resp}")
                                        current_ocr_result = f"ERROR: Response parsing failed - {e_resp}"
                                else:
                                    print(f"      ⚠️ Warning: Empty or invalid response received for {chunk_filename}.")
                                    current_ocr_result = "ERROR: Empty or invalid response from API"

                                # Write result to CSV
                                writer.writerow([current_ocr_result])
                                # print(f"      Wrote OCR result to CSV.")

                                # 8. Update History for Next Iteration
                                ocr_result_n_minus_2 = ocr_result_n_minus_1
                                ocr_result_n_minus_1 = current_ocr_result # Store the raw result from this chunk

                            except Exception as e_api:
                                print(f"    🛑 An error occurred during API call or processing for {chunk_filename}:")
                                traceback.print_exc() # Print full traceback for debugging
                                error_message = f"ERROR: API call failed for {chunk_filename} - {type(e_api).__name__}"
                                writer.writerow([error_message])
                                # Update history with error marker to avoid propagating bad context
                                ocr_result_n_minus_2 = ocr_result_n_minus_1
                                ocr_result_n_minus_1 = error_message # Store error marker

                            finally:
                                # 9. Delete Uploaded File from Gemini
                                if gemini_file:
                                    delete_gemini_file(gemini_file)
                                time.sleep(1.5) # Slightly increased delay


            except IOError as e:
                print(f"🛑 Error opening or writing to CSV file {OUTPUT_CSV_FILE}: {e}")
            except Exception as e_main:
                 print(f"🛑 An unexpected error occurred during the main process:")
                 traceback.print_exc()

            print("\n--- OCR Process Complete for Folder ---")
            print(f"Results saved in: {OUTPUT_CSV_FILE}") # Display the final generated path

Gemini API Key configured.
Starting OCR process for single folder: 'Langage et réalité (AGREG ext. 2018 - note : 19)'
  Subject detected: 'Langage et réalité'
  Output CSV will be saved as: '/content/Langage et réalité.csv'
  CSV file opened successfully.
  Wrote folder title to CSV: Langage et réalité (AGREG ext. 2018 - note : 19)
  Found 52 chunk(s) to process.


Processing Chunks:   0%|          | 0/52 [00:00<?, ?it/s]


    Processing chunk 1/52: 0001.png
      Using FIRST chunk system prompt.
      Uploading: 0001.png...
      Calling Gemini API...
      OCR result received (~945 chars).

    Processing chunk 2/52: 0002.png
      Using CONTEXT system prompt (Context length: ~1038 chars).
      Uploading: 0002.png...
      Calling Gemini API...
      OCR result received (~863 chars).

    Processing chunk 3/52: 0003.png
      Using CONTEXT system prompt (Context length: ~1902 chars).
      Uploading: 0003.png...
      Calling Gemini API...
      OCR result received (~440 chars).

    Processing chunk 4/52: 0004.png
      Using CONTEXT system prompt (Context length: ~1397 chars).
      Uploading: 0004.png...
      Calling Gemini API...
      OCR result received (~877 chars).

    Processing chunk 5/52: 0005.png
      Using CONTEXT system prompt (Context length: ~1411 chars).
      Uploading: 0005.png...
      Calling Gemini API...
      OCR result received (~837 chars).

    Processing chunk 6/52: 000